# 1. Configuración de entorno

En esta sección validamos que nuestro entorno de trabajo esté correctamente configurado antes de comenzar el análisis.  
Los pasos incluyen:

1. **Versión de Python**  
   - Se verifica que esté instalada la versión **3.11 o superior** (se recomienda 3.13).  
   - Esto garantiza compatibilidad con librerías modernas de análisis de datos y machine learning.

2. **Importación de librerías base**  
   - Se cargan librerías fundamentales:  
     - `numpy`, `pandas`: manipulación y análisis de datos.  
     - `matplotlib`, `seaborn`: visualización de datos.  
     - `scipy`: funciones estadísticas.  
   - Además se configuran estilos gráficos y opciones de visualización en pandas para trabajar con tablas más grandes.

3. **Verificación de versiones críticas**  
   - Se comprueba que `scikit-learn` esté instalado y en una versión **>= 1.0.1**.  
   - Esto es esencial ya que `scikit-learn` se usará para el modelado (baseline y posteriores).

Con esta configuración inicial aseguramos que el entorno sea reproducible y que todas las dependencias necesarias estén listas antes de continuar con el **EDA** y el **baseline**.


In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

assert sys.version_info >= (3, 11), "Este notebook trabajo con python 3.11 o superiores (recomendado 3.13)"

print(f"Python {sys.version_info.major}.{sys.version_info.minor} instalado correctamente")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
sns.set_palette("husl")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Librerías importadas exitosamente")

In [ ]:
# Verificar versiones de librerías críticas
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1"), "Requiere scikit-learn >= 1.0.1"
print(f"scikit-learn {sklearn.__version__} instalado")

# 2. Metodología CRISP-DM
## 2.1. Comprensión del Negocio
El problema de Google Play Store  

**Contexto:**  
Es 2025. El mercado de aplicaciones móviles es altamente competitivo: millones de apps conviven en Google Play Store.  
Los desarrolladores buscan mejorar la visibilidad de sus aplicaciones y los usuarios dependen del **rating promedio** para decidir qué descargar.  

**Problema actual:**  
- El rating se conoce **solo después** de que los usuarios descargan y reseñan.  
- Las valoraciones son **altamente variables** y pueden depender de múltiples factores (categoría, descargas, precio, tamaño, tipo de app).  
- Los desarrolladores carecen de una herramienta para **estimar la calificación potencial** de una app antes o durante su lanzamiento.  
- La competencia es muy alta: una diferencia de décimas en rating puede significar miles de descargas menos.  

---

### 2.1.1. Solución propuesta  
Construir un **sistema automático de predicción de rating** de apps a partir de sus características disponibles en el dataset de Google Play Store.  

---

### 2.1.2. Definiendo el éxito  

**Métrica de negocio:**  
- Ayudar a los desarrolladores a anticipar la valoración probable de su app.  
- Reducir la dependencia de pruebas de mercado costosas o lentas.  
- Identificar características clave que favorecen una alta valoración (≥ 4.3).  

**Métrica técnica:**  
- Lograr un **Error Absoluto Medio (MAE) < 0.5 estrellas** en la predicción de rating.  
- Para la versión de clasificación (alta vs. baja calificación): obtener un **F1-score > 0.70**.  

**¿Por qué estos valores?**  
- El rating va de 1 a 5 → un error de 0.5 equivale a 10% de la escala.  
- Una diferencia de medio punto puede marcar la visibilidad de la app en el ranking.  
- Tasadores humanos (usuarios) también muestran variabilidad similar en sus calificaciones.  

---

### 2.1.3 Preguntas críticas antes de empezar  

1. **¿Realmente necesitamos ML?**  
   - Alternativa 1: Calcular el promedio de ratings por categoría → demasiado simple, no captura variabilidad.  
   - Alternativa 2: Reglas heurísticas (ej. “si es gratis y tiene muchas descargas, tendrá rating alto”) → insuficiente.  
   - **Conclusión:** Sí, ML es apropiado para capturar relaciones no lineales y múltiples factores.  

2. **¿Qué pasa si el modelo falla?**  
   - Transparencia: aclarar que es una estimación automática.  
   - Complementar con rangos de predicción (ej: intervalo de confianza).  
   - Mantener como referencia comparativa, no como único criterio de éxito.  

3. **¿Cómo mediremos el impacto?**  
   - Capacidad de anticipar apps con alta probabilidad de éxito.  
   - Ahorro de tiempo en validaciones preliminares.  
   - Insights para desarrolladores sobre qué factores influyen más en el rating.  

---


## 2.2. Comprensión de los Datos  

El objetivo de esta fase es explorar y entender el dataset de Google Play Store antes de construir modelos **(análisis exploratorio)**.  
Nos centraremos en:  

1. **Vista rápida del dataset**  
   - Identificar dimensiones (filas × columnas).  
   - Tipos de datos (numéricos, categóricos, texto, fechas).  
   - Valores faltantes obvios y rangos sospechosos.

2. **Descripción de variables**  
   - Revisar cada columna y entender su significado.  
   - Detectar qué variables podrían ser útiles como predictores y cuál será la variable objetivo (rating).  

3. **Detección de problemas en los datos**  
   - Análisis de valores faltantes.  
   - Estrategias: eliminar filas/columnas, imputar valores o crear indicadores de “dato faltante”.  

4. **Estadísticas descriptivas y univariadas**  
   - Media vs mediana (sesgo de la distribución).  
   - Desviación estándar (variabilidad, posibles outliers).  
   - Mínimos/máximos sospechosos.  
   - Histogramas para ver forma (normal, sesgada, bimodal, uniforme, picos extraños).  

5. **Análisis de variables categóricas**  
   - Distribución de categorías (ej. categorías de apps, tipo de app, content rating).  
   - Detección de clases dominantes o categorías poco representadas.  

6. **Correlaciones y relaciones entre variables**  
   - Matriz de correlación de Pearson para variables numéricas.  
   - Identificar relaciones fuertes, moderadas o débiles.  
   - Importante: recordar que **correlación ≠ causalidad**.  
7. **Análisis de outliers**
   -  Tipos e identificación de outliers a través de diferentes métodos.

---

**Nota:**  
No siempre es necesario aplicar todos los pasos con igual profundidad.  
- Para este proyecto, el foco está en **identificar variables relevantes para predecir el rating** y **limpiar datos inconsistentes**.  
- Otros análisis más complejos (ej. NLP sobre descripciones) se pueden dejar como trabajo futuro (según trabajos de referencia investigados).

---


### 2.2.1 Descarga de datos  

En este paso descargamos el dataset de Google Play Store desde Kaggle y lo organizamos en la estructura de carpetas del proyecto.  

1. Usamos la librería `kagglehub` para acceder al dataset público **`lava18/google-play-store-apps`** directamente desde Kaggle.  
2. Se define una ruta clara dentro del proyecto para almacenar los datos originales: `../data/original/google-play-store/`. Esto ayuda a mantener la reproducibilidad y una estructura organizada.  
3. Con la función `shutil.copytree` copiamos los archivos descargados a la carpeta destino. De esta forma, el dataset queda disponible en nuestro directorio de trabajo para su análisis posterior.  


In [ ]:
import kagglehub
import shutil

def download_data(origin_repository, target_folder):
    # Descargar dataset
    path = kagglehub.dataset_download(origin_repository)
    
    # Copiar los archivos descargados
    shutil.copytree(path, target_folder, dirs_exist_ok=True)
    

download_data("lava18/google-play-store-apps", "../../data/original/google-play-store")

### 2.2.2 Carga de datos  

En este paso realizamos la **lectura del archivo CSV** que contiene el dataset descargado previamente.  

- Definimos una función `load_data(path, file)` que recibe la ruta y el nombre del archivo, y lo carga con `pandas.read_csv()`.  
- Cargamos el dataset principal en la variable `applications_data` desde la carpeta `../data/original/google-play-store/`.  
- Incluimos una verificación simple:  
  - Si el dataset se carga con éxito, se imprime `"Dataset loaded"`.  
  - En caso contrario, se muestra un mensaje de error.  

Con esta validación aseguramos que el archivo esté disponible y correctamente leído antes de continuar con el análisis exploratorio.


In [ ]:
import pandas as pd
import numpy as np

def load_data(path, file):
    return pd.read_csv(f"{path}/{file}")

def convert_numeric_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convierte a numéricas solo las columnas que deberían serlo, sin tocar 'Size'.
    Usa to_numeric(errors='coerce') para evitar ValueError si aparece texto.
    """
    df = df.copy()

    # Rating
    if "Rating" in df.columns:
        df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")

    # Reviews: quitar comas y cualquier carácter no numérico/punto
    if "Reviews" in df.columns:
        df["Reviews"] = (
            df["Reviews"].astype(str)
            .str.replace(r"[^\d.]", "", regex=True)
            .pipe(pd.to_numeric, errors="coerce")
        )

    # Installs: quitar +, comas y cualquier carácter no numérico/punto
    if "Installs" in df.columns:
        df["Installs Numeric"] = (
            df["Installs"].astype(str)
            .str.replace(r"[^\d.]", "", regex=True)
            .pipe(pd.to_numeric, errors="coerce")
        )

    # Price: quitar $ y cualquier carácter no numérico/punto
    if "Price" in df.columns:
        df["Price"] = (
            df["Price"].astype(str)
            .str.replace(r"[^\d.]", "", regex=True)
            .pipe(pd.to_numeric, errors="coerce")
        )


    if "Size" in df.columns:
        def parse_size(x):
            if isinstance(x, str):
                x = x.strip()
                if x.endswith("M"):
                    return float(x[:-1])
                elif x.endswith("k") or x.endswith("K"):
                    return float(x[:-1]) / 1024  # KB -> MB
                else:
                    return np.nan
            return np.nan
        df["Size"] = df["Size"].apply(parse_size)

    return df


temp_applications_data = load_data("../../data/original/google-play-store", "googleplaystore.csv")
applications_data = convert_numeric_columns(temp_applications_data)


if len(applications_data):
    print("Dataset cargado")
else:
    print("Error cargando dataset")


### 2.2.3 Vista rápida del dataset

**Dimensiones y columnas**
- Registros: **10,841** filas.
- Columnas: actualmente **14**; **originalmente eran 13** y se **añadió** una columna derivada: **`Installs Numeric`** para análisis con describe.
- Memoria aproximada: **~1.2 MB**.

**Tipos de datos (y transformaciones realizadas)**
- Numéricas (`float64`): `Rating`, `Reviews`, `Size`, `Price`, **`Installs Numeric`**.
- Categóricas / texto (`object`): `App`, `Category`, `Installs` *(forma original con “1,000+”)*, `Type`, `Content Rating`, `Genres`, `Last Updated`, `Current Ver`, `Android Ver`.
- Transformaciones ya aplicadas:
  - **`Installs`** se **conservó** en su formato original (categórico con “+” y comas) **y** se creó **`Installs Numeric`** mapeando esos rangos a números (0 … 1,000,000,000).
  - **`Price`**, **`Reviews`** y **`Size`** fueron normalizadas/parseadas a **numérico** para análisis y modelado.

**Valores faltantes (no-null count → faltantes aprox.)**
- `Rating`: 9,367 → **1,474 faltantes (~13.6%)**.
- `Size`: 9,145 → **1,696 faltantes (~15.6%)**.
- `Current Ver`: 10,833 → **8 faltantes (~0.07%)**.
- `Android Ver`: 10,838 → **3 faltantes (~0.03%)**.
- `Content Rating`: 10,840 → **1 faltante (~0.01%)**.
- `Price`: 10,840 → **1 faltante (~0.01%)**.
- `Installs Numeric`: 10,840 → **1 faltante (~0.01%)**.
- Resto de columnas: **sin faltantes**.

**Duplicados:**
-   Se identificaron **483 filas duplicadas** (≈ **4.46%** del
    dataset).\
-   Ejemplos de duplicados incluyen apps como:
    -   *Quick PDF Scanner + OCR FREE*\
    -   *Box*\
    -   *Google My Business*\
    -   *ZOOM Cloud Meetings*\
    -   *join.me -- Simple Meetings*\

**Rangos y valores sospechosos (según `describe()`)**
- `Rating`: **min = 1.0**, **max = 19.0** → **19** es inválido para la escala 1–5 (error de dato a corregir).
- `Reviews`: media ~ **444k**, **p75 ≈ 54,768**, **max ≈ 78M** → valores altos plausibles; tratar como **outliers**.
- `Size` (MB): media ~ **21.5**, **p50 = 13**, **p75 = 30**, **max = 100** → distribución sesgada a la derecha; mínimos muy bajos (**0.01**) a revisar.
- `Price` (USD): **mediana = 0** y **p75 = 0** → la mayoría son **apps gratuitas**; **max = 400** sugiere outliers de precio.
- `Installs Numeric`: **p25 = 1,000**, **p50 = 100,000**, **p75 = 5,000,000**, **max = 1,000,000,000** → escala muy amplia; conviene usar **transformaciones log** o **binning** en el EDA/modelado.

**Conclusión inicial**
- Los **faltantes** más relevantes están en `Rating` y `Size`; habrá que decidir estategía para aumentar, imputar o nivelar los datos.
- Existen **outliers (no legítimos)** (ej. `Rating = 19`) y variables con **colas largas** (ej. `Reviews`, `Installs Numeric`, `Price`).
- Eliminar **duplicados** para evitar sesgos de análisis y que no introduzcan ruidos.


In [ ]:
print("=" * 80)
print("INFORMACIÓN GENERAL DEL DATASET".center(80))
print("=" * 80)

display(applications_data.head().style.background_gradient(cmap='RdYlGn', subset=['Rating']))

# Información detallada
print("\n" + "=" * 80)
print("ESTRUCTURA DE DATOS".center(80))
print("=" * 80)
applications_data.info()

# Estadísticas descriptivas
print("\n" + "=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS".center(80))
print("=" * 80)
display(applications_data.describe().round(2).T)

print("\n" + "=" * 80)
print("DATOS DUPLICADOS".center(80))
print("=" * 80)

# Contar duplicados
num_duplicados = applications_data.duplicated().sum()
print(f"Total de registros duplicados: {num_duplicados}")

# Mostrar ejemplos de duplicados si existen
if num_duplicados > 0:
    print("\nEjemplos de filas duplicadas:\n")
    display(applications_data[applications_data.duplicated()].head())
else:
    print("No se encontraron registros duplicados.")


### 2.2.4 Descripción de las variables

In [ ]:
variables = [
    'App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price',
    'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver',
    'Installs Numeric'
]

tipos = [
    'Categórica',          # App
    'Categórica',          # Category
    'Numérica (Target)',   # Rating
    'Numérica',            # Reviews
    'Numérica (MB)',       # Size
    'Categórica (rango)',  # Installs
    'Categórica',          # Type
    'Numérica (USD)',      # Price
    'Categórica',          # Content Rating
    'Categórica',          # Genres
    'Texto (fecha)',       # Last Updated (parseable a fecha)
    'Texto',               # Current Ver
    'Texto',               # Android Ver
    'Numérica',            # Installs Numeric
]

descripciones = [
    'Nombre de la aplicación.',
    'Categoría oficial de la app en Google Play.',
    'Calificación promedio de usuarios (1 a 5).',
    'Número de reseñas reportadas.',
    'Tamaño aproximado de la app en MB.',
    'Instalaciones en rango (p.ej., "1,000+").',
    'Tipo de app (Free / Paid).',
    'Precio en USD (0 para gratuitas).',
    'Clasificación de contenido (Everyone, Teen, etc.).',
    'Género(s) de la app.',
    'Fecha de última actualización (texto en origen).',
    'Versión actual declarada por el desarrollador.',
    'Versión mínima de Android requerida.',
    'Instalaciones convertidas a número para análisis.'
]

valores_faltantes = [applications_data[col].isnull().sum() if col in applications_data.columns else None for col in variables]

metadata = {
    'Variable': variables,
    'Tipo': tipos,
    'Descripción': descripciones,
    'Valores Faltantes': valores_faltantes
}

df_metadata = pd.DataFrame(metadata)

# Mostrar con resaltado de faltantes
styled = df_metadata.style.applymap(
    lambda x: 'background-color: #ffcccc' if isinstance(x, (int, float)) and x > 0 else '',
    subset=['Valores Faltantes']
)

display(styled)

# Resumen de dtypes originales (informativo)
dtypes_resumen = applications_data[variables].dtypes.astype(str).reset_index()
dtypes_resumen.columns = ['Variable', 'dtype pandas']
display(dtypes_resumen)

### 2.2.5 Detección de problemas en los datos 

**Resumen de hallazgos (valores faltantes):**
- `Size` ≈ 15.6% y `Rating` ≈ 13.6% concentran la mayoría de los faltantes.
- Faltantes puntuales (≈0.01%): `Type`, `Price`, `Content Rating`, `Installs Numeric` ocurren en la misma(s) fila(s) → patrón conjunto.
- `Android Ver` (0.03%) y `Current Ver` (0.07%) con faltantes residuales, parcialmente correlacionados con el grupo anterior.

**Heatmap de correlación de patrones de faltantes (interpretación):**
- Correlación 1.00 entre `Type`, `Price`, `Content Rating`, `Installs Numeric`: las ausencias co-ocurren en el/los mismos registros. Acciones coordinadas.
- `Android Ver` muestra correlación moderada (~0.58) con ese grupo: algunas veces falta junto con ellos.
- `Size`, `Rating`, `Current Ver` tienen patrones de faltantes independientes del grupo anterior (correlaciones cercanas a 0), lo que sugiere causas distintas.

#### Posibles estrategias de corrección

- Limpieza básica
  - Eliminar duplicados (483 filas) para evitar sesgos.
  - Validar y corregir outliers imposibles, p. ej., `Rating = 19` → convertir a NaN para tratarlo como faltante.

- Imputación (conservadora y por grupos)
  - `Rating` (target): para modelado, eliminar filas sin `Rating`; para EDA descriptivo, imputar mediana por `Category` solo para visualización.
  - `Size`: imputar mediana por `Category × Type` y crear indicador `size_missing`.
  - `Android Ver`, `Current Ver`: imputar moda por `Category` y crear indicadores `androidver_missing`, `currentver_missing`.
  - Faltantes conjuntos (`Type`, `Price`, `Content Rating`, `Installs Numeric`):
    - Si es 1 fila: eliminarla es lo más simple y seguro.
    - Alternativa (si se prefiere imputar):
      - `Type`: inferir desde `Price` (0 → Free, >0 → Paid).
      - `Price`: 0 si `Type == Free`, si `Paid` usar mediana por `Category`.
      - `Content Rating`: moda por `Category`.
      - `Installs Numeric`: mediana por `Category × Type` o por bin de `Installs`.



#### Estrategias de “nivelación” según los porcentajes observados

- Size (~15.6% faltantes, >5% y <<60%)
  - Acción: imputar mediana por grupo `Category × Type`.
  - Añadir flag: `size_missing = 1` cuando falte (conserva señal de ausencia).
  - Justificación: volumen relevante; la mediana por grupos respeta diferencias entre tipos/categorías.

- Rating (~13.6% faltantes, >5% y <<60%) [variable objetivo]
  - Para modelado: eliminar filas sin `Rating` (evita sesgo por imputación del target).
  - Para EDA descriptivo: si se requiere visualizar completos, imputar mediana por `Category` solo para gráficos/tablas (no para entrenamiento).
  - Justificación: imputar el target puede distorsionar métricas.

- Current Ver (0.07%) y Android Ver (0.03%) (<5%)
  - Acción: imputar con la moda por `Category`. Flags opcionales `currentver_missing` y `androidver_missing`.
  - Justificación: impacto ínfimo; moda es suficiente y estable.

- Faltantes “en bloque” en la misma fila: Type, Price, Content Rating, Installs Numeric (≈0.01% cada uno; correlación 1.00)
  - Si es 1 fila: eliminarla directamente.
  - Si hubiera más en el futuro y se prefiriera imputar coordinadamente:
    - `Type` desde `Price` (0 → Free, >0 → Paid),
    - `Price` = 0 si `Free`, si `Paid` usar mediana por `Category`,
    - `Content Rating` = moda por `Category`,
    - `Installs Numeric` = mediana por `Category × Type`.
  - Justificación: co-ocurren; eliminar 1 fila no afecta el conjunto y evita inconsistencias.

- Transformaciones para estabilizar distribuciones (complementarias a la imputación)
  - `Reviews` y `Installs Numeric`: aplicar `log1p` para análisis y futuros modelos. *****************************
  - `Installs` (rangos): tratar como ordinal/bins en el EDA.

- Limpieza previa necesaria
  - Eliminar duplicados (483 filas).
  - Corregir valores imposibles detectados en el EDA (ej. `Rating = 19` → NaN) y re-entrar al flujo de imputación/nivelación anterior.

In [ ]:
def analyze_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """Análisis completo de valores faltantes con visualizaciones."""
    missing_counts = df.isnull().sum()
    missing_pct = (missing_counts / len(df)) * 100

    missing_df = pd.DataFrame({
        'Columna': df.columns,
        'Valores_Faltantes': missing_counts.values,
        'Porcentaje': missing_pct.values,
        'Tipo_Dato': df.dtypes.values
    })

    missing_df = missing_df[missing_df['Valores_Faltantes'] > 0].sort_values('Porcentaje', ascending=False)

    if len(missing_df) == 0:
        print("No hay valores faltantes en el dataset")
        return missing_df

    # Visualización: barras y correlación de patrones de faltantes
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

    # Gráfico de barras de % faltantes
    ax1.bar(missing_df['Columna'], missing_df['Porcentaje'], color='coral')
    ax1.set_xlabel('Columna')
    ax1.set_ylabel('Porcentaje de Valores Faltantes (%)')
    ax1.set_title('Valores Faltantes por Columna')
    ax1.axhline(y=5, color='r', linestyle='--', label='Umbral 5%')
    ax1.axhline(y=60, color='purple', linestyle='--', label='Umbral 60%')
    ax1.tick_params(axis='x', rotation=90)
    ax1.legend()

    # Heatmap de correlación de patrones de faltantes
    mask_df = df[missing_df['Columna'].tolist()].isnull().astype(int)
    if mask_df.shape[1] >= 2:
        corr = mask_df.corr()
        sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', vmin=-1, vmax=1, ax=ax2)
        ax2.set_title('Correlación de Patrones de Valores Faltantes')
    else:
        ax2.axis('off')
        ax2.set_title('Correlación de faltantes (no aplica: 1 columna)')

    plt.tight_layout()
    plt.show()

    return missing_df

missing_analysis = analyze_missing_values(applications_data)
if missing_analysis is not None and not missing_analysis.empty:
    display(missing_analysis)

### 2.2.6 Estadisticas descriptivas y univariadas (númerico)

A partir de la tabla de estadísticas y los gráficos generados para `Rating`, `Reviews`, `Size`, `Price` e `Installs Numeric`, se observan los siguientes puntos clave.

- Rating
  - Media ≈ 4.19 y mediana ≈ 4.30 → ligera cola a la izquierda (más apps con rating alto). Hay un valor imposible (≈19), confirmado en el boxplot/Q-Q como outlier extremo.
  - Outliers: ~5% por IQR, dominados por el valor inválido y algunos ratings bajos.
  - Q-Q plot: desviación frente a normalidad, esperable para una variable acotada [1,5].
  - Implicación/acción: eliminar filas sin `Rating` para modelado; corregir `Rating=19 → NaN` y excluir; no aplicar transformaciones (la escala es ya interpretables).

- Reviews
  - Media ≫ mediana (pico en 0–pocos miles; máximo ≈ 78M) → cola muy larga a la derecha.
  - Boxplot: ~18% outliers por IQR (muchas apps con reseñas muy altas).
  - Q-Q plot: gran desviación de normalidad (heavy tail).
  - Relación con Rating: correlación positiva muy débil (~0.07), tendencia casi plana.
  - Implicación/acción: usar `log1p(Reviews)` para estabilizar la distribución en análisis/modelado; considerar winsorizar p99.9 para vistas tabulares si se desea.

- Size (MB)
  - Media > mediana (≈ 21.5 vs 13) → sesgo a la derecha; valores hasta 100 MB.
  - ~6% outliers por IQR, especialmente en colas altas.
  - Q-Q plot: curvatura en colas; no normal.
  - Relación con Rating: correlación positiva débil (~0.08); señal muy tenue.
  - Implicación/acción: imputar faltantes por `Category × Type` y añadir `size_missing`; opcionalmente probar `log1p(Size)` o binning para robustecer.

- Price (USD)
  - Mediana = 0 (mayoría gratis) y cola a la derecha con máximos altos (≈ 400).
  - ~7% outliers por IQR; Q-Q muestra heavy tail.
  - Relación con Rating: correlación negativa muy débil (~-0.02).
  - Implicación/acción: crear `is_free = (Price == 0)` y, si se usa `Price` continuo, considerar `log1p(Price)` para las pocas apps pagas; validar coherencia `Type=Free ⇒ Price=0`.

- Installs Numeric
  - Media ≫ mediana (100k) con máximo 1e9 → distribución extremadamente sesgada a la derecha.
  - ~7–8% outliers por IQR; Q-Q muy alejado de normalidad.
  - Relación con Rating: correlación débil positiva (~0.05) y tendencia casi plana.
  - Implicación/acción: usar `log1p(Installs Numeric)` o bins ordinales para análisis; verificar coherencia con `Installs` textual.

Recomendaciones transversales
- Eliminar duplicados antes de resumir para evitar sesgos.
- Tratar outliers evidentes no-legítimos (p. ej. `Rating=19`). Para colas largas legítimas (`Reviews`, `Installs Numeric`, `Price`): preferir `log1p` o winsorización solo para visualizaciones.
- Mantener consistencia: `Type=Free ⇒ Price=0`; `Installs Numeric` coherente con el rango de `Installs`.
- Para relaciones con `Rating`, las correlaciones lineales observadas son débiles; la señal puede emerger mejor con interacciones (p. ej., `is_free × installs_bin`) o modelos no lineales.



In [ ]:
from scipy import stats

# Selección de columnas numéricas relevantes
numeric_cols = [c for c in ['Rating', 'Reviews', 'Size', 'Price', 'Installs Numeric'] if c in applications_data.columns]

# Tabla de estadísticas básicas (media, mediana, std, min, p25, p50, p75, max)
describe_tbl = applications_data[numeric_cols].describe(percentiles=[0.25, 0.5, 0.75]).T

# Métricas adicionales robustas
extra = pd.DataFrame(index=numeric_cols)
extra['mad'] = [stats.median_abs_deviation(applications_data[c].dropna()) for c in numeric_cols]
extra['skew'] = [applications_data[c].skew(skipna=True) for c in numeric_cols]
extra['kurtosis'] = [applications_data[c].kurtosis(skipna=True) for c in numeric_cols]
extra['cv'] = [applications_data[c].std(skipna=True) / applications_data[c].mean(skipna=True) if applications_data[c].mean(skipna=True) not in [0, np.nan] else np.nan for c in numeric_cols]

stats_table = describe_tbl.join(extra)
display(stats_table.round(3))


def univariate_analysis(df: pd.DataFrame, column: str, target: str | None = None):
    """Análisis univariado con histograma, boxplot, Q-Q plot y relación con target."""
    series = df[column].dropna()
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # 1) Histograma con líneas de media y mediana
    ax1 = axes[0, 0]
    ax1.hist(series, bins=50, edgecolor='black', alpha=0.7)
    ax1.axvline(series.mean(), color='red', linestyle='--', label=f"Media: {series.mean():.2f}")
    ax1.axvline(series.median(), color='green', linestyle='--', label=f"Mediana: {series.median():.2f}")
    ax1.set_title(f"Distribución de {column}")
    ax1.set_xlabel(column)
    ax1.set_ylabel('Frecuencia')
    ax1.legend()
    ax1.grid(alpha=0.3)

    # 2) Boxplot + conteo de outliers (IQR)
    ax2 = axes[0, 1]
    bp = ax2.boxplot(series, vert=True, patch_artist=True)
    bp['boxes'][0].set_facecolor('lightblue')
    Q1, Q3 = series.quantile(0.25), series.quantile(0.75)
    IQR = Q3 - Q1
    outliers_mask = (series < Q1 - 1.5 * IQR) | (series > Q3 + 1.5 * IQR)
    n_out = int(outliers_mask.sum())
    pct_out = 100 * n_out / len(series) if len(series) else 0
    ax2.set_title(f"Boxplot de {column}")
    ax2.set_ylabel(column)
    ax2.grid(alpha=0.3)
    ax2.text(1.1, Q3, f"Outliers: {n_out} ({pct_out:.1f}%)", fontsize=10)

    # 3) Q-Q plot normal
    ax3 = axes[1, 0]
    stats.probplot(series, dist='norm', plot=ax3)
    ax3.set_title('Q-Q Plot (Normalidad)')
    ax3.grid(alpha=0.3)

    # 4) Relación con target si aplica
    ax4 = axes[1, 1]
    if target is not None and target in df.columns and column != target:
        valid = df[[column, target]].dropna()
        ax4.scatter(valid[column], valid[target], alpha=0.4, s=10)
        ax4.set_xlabel(column)
        ax4.set_ylabel(target)
        ax4.set_title(f"{column} vs {target}")
        # Línea de tendencia (ajuste lineal simple)
        if len(valid) > 1:
            z = np.polyfit(valid[column], valid[target], 1)
            p = np.poly1d(z)
            xs = np.linspace(valid[column].min(), valid[column].max(), 200)
            ax4.plot(xs, p(xs), 'r--', alpha=0.8, label='Tendencia')
            corr = valid[column].corr(valid[target])
            ax4.text(0.05, 0.95, f"Correlación: {corr:.3f}", transform=ax4.transAxes,
                     fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat'))
            ax4.legend()
    else:
        ax4.axis('off')
        ax4.grid(alpha=0.3)

    plt.suptitle(f"Análisis Univariado: {column}", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Ejecutar análisis univariado para cada métrica numérica, relacionando con Rating
for col in numeric_cols:
    univariate_analysis(applications_data, col, target='Rating')

### 2.2.7. Análisis Univariado Categórico
- Category
  - Distribución: alta concentración en `FAMILY` (~19%) y `GAME` (~12%). El resto de categorías tienen menor peso individual; el grupo `Others` acumula ~31% del total.
  - Rating por categoría: diferencias moderadas; la **mediana** suele estar entre 4.2–4.4. Algunas categorías muestran desviación estándar mayor (p. ej., `PRODUCTIVITY`, `LIFESTYLE`), indicando más variabilidad de valoración.
  - Implicaciones: riesgo de sesgo por categorías mayoritarias en análisis agregados. Para modelado, conviene usar dummies Top-K o codificación ordinal/target encoding con cuidado (evitar fuga). Agrupar colas largas en `Others` es adecuado para visualización.

- Content Rating
  - Distribución: `Everyone` domina (~79%), seguido por `Teen` (~12%); `Mature 17+` y `Everyone 10+` suman ~9% en conjunto; clases raras casi nulas.
  - Rating por nivel de contenido: medias similares (≈4.1–4.3). `Teen` tiende a mediana 4.3 y variabilidad algo menor; `Mature 17+` muestra algo más de dispersión.
  - Implicaciones: por el fuerte desbalance, esta variable aporta señal limitada por sí sola. Útil como interacción con `Category`/`Genres`.

- Type
  - Distribución: `Free` ≈ 93%, `Paid` ≈ 7% (clase muy desbalanceada); existe un registro anómalo (valor 0) en los gráficos que debe eliminarse/corregirse.
  - Rating por tipo: medias muy cercanas (Free ≈ 4.19, Paid ≈ 4.27). La diferencia es pequeña y probablemente no significativa sin controlar otras variables (p. ej., `Category`).
  - Implicaciones: por el desbalance extremo, conviene usar `is_free` como binaria y, si se modela interacción con `Installs` o `Price`, puede emerger señal. Validar regla `Type=Free ⇒ Price=0`.

- Genres Main (primer género)
  - Distribución: gran cola larga; `Others` concentra ~48%. Entre Top-12, `Tools`, `Entertainment` y `Education` destacan en frecuencia.
  - Rating por género: diferencias pequeñas (medianas ~4.2–4.4), con algunas variaciones en dispersión (p. ej., `Medical` y `Lifestyle` más variables).
  - Implicaciones: por la alta cardinalidad y colas largas, mantener Top-K + `Others` en EDA ayuda a la legibilidad. Para modelado, preferir codificación que reduzca dimensionalidad (Top-K dummies, hashing, o target encoding con validación adecuada).

In [ ]:
def analyze_categorical_compact(df: pd.DataFrame, cat_col: str, target_col: str, top_n: int = 12):
    """
    Versión compacta para variables con muchas categorías:
    - Ordena por frecuencia, muestra top_n y agrupa el resto en "Others".
    - Barras horizontales, pie chart compacto, boxplot y tabla para top_n.
    """
    data = df[[cat_col, target_col]].dropna(subset=[cat_col, target_col]).copy()
    if data.empty:
        print(f"Sin datos para {cat_col} y {target_col}")
        return

    counts = data[cat_col].value_counts()
    top_cats = counts.head(top_n)
    others_count = counts.iloc[top_n:].sum()

    # Mapeo a top_n + Others
    mapping = {c: c for c in top_cats.index}
    data['__cat__'] = data[cat_col].where(data[cat_col].isin(top_cats.index), other='Others')

    # Recalcular conteos con Others
    counts_compact = data['__cat__'].value_counts()
    order = list(top_cats.index) + (['Others'] if 'Others' in counts_compact.index else [])

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # 1) Barras horizontales (mejor legibilidad)
    ax1 = axes[0, 0]
    vals = counts_compact.loc[order]
    ax1.barh(range(len(vals)), vals.values, color=plt.cm.Set3(range(len(vals))))
    ax1.set_yticks(range(len(vals)))
    ax1.set_yticklabels(order)
    ax1.invert_yaxis()
    ax1.set_title(f'Distribución (Top {top_n}) de {cat_col}')
    ax1.set_xlabel('Frecuencia')
    for i, v in enumerate(vals.values):
        ax1.text(v, i, f'  {v} ({v/len(data)*100:.1f}%)', va='center')

    # 2) Pie chart compacto
    ax2 = axes[0, 1]
    ax2.pie(vals.values, labels=order, autopct='%1.1f%%', startangle=140,
            colors=plt.cm.Set3(range(len(vals))))
    ax2.set_title(f'Proporción (Top {top_n} + Others) de {cat_col}')

    # 3) Boxplot del target por categoría (solo top_n)
    ax3 = axes[1, 0]
    top_mask = data['__cat__'] != 'Others'
    data_top = data[top_mask]
    data_top.boxplot(column=target_col, by='__cat__', ax=ax3)
    ax3.set_title(f'{target_col} por {cat_col} (Top {top_n})')
    ax3.set_xlabel(cat_col)
    ax3.set_ylabel(target_col)
    plt.sca(ax3)
    plt.xticks(rotation=30, ha='right')

    # 4) Tabla de estadísticas por categoría (solo top_n y Others si existe)
    ax4 = axes[1, 1]
    ax4.axis('off')
    stats_by_cat = data.groupby('__cat__')[target_col].agg(['count', 'mean', 'median', 'std']).loc[order].round(2)
    table = ax4.table(cellText=stats_by_cat.reset_index().values,
                      colLabels=['Categoría', 'N', 'Media', 'Mediana', 'Desv.Est.'],
                      cellLoc='center', loc='center', colWidths=[0.35, 0.12, 0.16, 0.16, 0.16])
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.05, 1.25)
    for i in range(5):
        table[(0, i)].set_facecolor('#40E0D0')
        table[(0, i)].set_text_props(weight='bold')

    plt.suptitle(f'Análisis Categórico Compacto: {cat_col}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Ejecutar la versión compacta para las categóricas clave
for cat in [c for c in ['Category', 'Content Rating', 'Type', 'Genres Main', 'Installs'] if c in applications_data.columns]:
    analyze_categorical_compact(applications_data, cat, 'Rating', top_n=12)

### 2.2.8. Análisis de correlación entre variables

#### 2.2.8.1. Variables con Mayor Relación
- Existe una fuerte correlación positiva entre **Installs Numeric** y **Reviews**:
  - Pearson: 0.64 (relación lineal moderada-fuerte).
  - Spearman: 0.97 (relación monótonica muy fuerte).
- Esto implica que a mayor número de instalaciones, mayor número de reseñas.

#### 2.2.8.2. Correlación de Pearson
- En general, las correlaciones de Pearson muestran relaciones más débiles que Spearman, lo cual indica que las relaciones lineales no son tan marcadas.
- **Installs Numeric y Reviews** presentan la correlación lineal más alta (0.64), siendo moderada-fuerte.
- **Size y Reviews** muestran una correlación positiva baja/Débil (0.24).
- El resto de variables (Rating, Price) tienen correlaciones casi nulas con las demás, lo que refleja poca relación lineal.

#### 2.2.8.3. Correlación de Spearman
- **Installs Numeric y Reviews** tienen la correlación más fuerte (0.97).
- **Size** muestra correlación moderada con **Reviews** (0.37) y con **Installs Numeric** (0.35).
- **Price** presenta correlaciones negativas con **Reviews** (-0.17) e **Installs Numeric** (-0.24).

#### 2.2.8.4. Observaciones Clave
- El número de instalaciones y las reseñas son las variables más relacionadas, lo cual es lógico, ya que más usuarios generan más interacciones.
- El tamaño de la aplicación influye ligeramente en reseñas e instalaciones, pero no de forma determinante.
- El precio no solo carece de relación positiva, sino que parece tener un impacto negativo sobre la popularidad (menos instalaciones y reseñas).

#### 2.2.8.5. Conclusión
- **Installs Numeric** y **Reviews** son las métricas más críticas en el dataset de **Google Play Store**, ya que reflejan el éxito y la popularidad de la aplicación.
- **Size** es un factor secundario con cierta relación.

In [ ]:
# Análisis de correlación mejorado para el proyecto de Google Play Store
def correlation_analysis(df):
    """Análisis de correlación con múltiples métricas"""
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    fig, axes = plt.subplots(1, 2, figsize=(20, 6))
    
    # 1. Correlación de Pearson
    corr_pearson = df[numeric_cols].corr(method='pearson')
    mask = np.triu(np.ones_like(corr_pearson), k=1)
    sns.heatmap(corr_pearson, mask=mask, annot=True, fmt='.2f', 
               cmap='coolwarm', center=0, ax=axes[0],
               vmin=-1, vmax=1, cbar_kws={"shrink": 0.8})
    axes[0].set_title('Correlación de Pearson (Lineal)')
    
    # 2. Correlación de Spearman  
    corr_spearman = df[numeric_cols].corr(method='spearman')
    sns.heatmap(corr_spearman, mask=mask, annot=True, fmt='.2f',
               cmap='coolwarm', center=0, ax=axes[1],
               vmin=-1, vmax=1, cbar_kws={"shrink": 0.8})
    axes[1].set_title('Correlación de Spearman (Monotónica)')
    
    plt.suptitle('Análisis de Correlación Multi-métrica', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Tabla de correlaciones importantes
    print("\n🔗 Correlaciones Significativas:")
    print("=" * 50)
    for method, corr_matrix in zip(['Pearson', 'Spearman'], [corr_pearson, corr_spearman]):
        print(f"\n{method}:")
        significant_corr = corr_matrix[(abs(corr_matrix) > 0.3) & (corr_matrix != 1)].stack()
        for (var1, var2), corr in significant_corr.items():
            strength = "Fuerte" if abs(corr) > 0.5 else "Moderada" if abs(corr) > 0.3 else "Débil"
            direction = "Positiva" if corr > 0 else "Negativa"
            print(f"  • {var1} y {var2}: {corr:+.3f} ({strength} {direction})")
    
# Ejecutar el análisis de correlación
correlation_analysis(applications_data)

#### 2.2.9. Análisis de Outliers (IQR, Z-Score e Isolation Forest)
**Resumen cuantitativo**
- Total de registros analizados: **10,841**.
- Filas marcadas como outlier por método:
  - IQR: **3,489** filas (32.18%) → refleja colas largas especialmente en `Reviews`, `Installs Numeric`, `Price`.
  - Z-Score (> |3|): **654** filas (6.03%) → mucho más selectivo, captura extremos verdaderamente alejados tras estandarización.
  - Isolation Forest (contaminación=10%): **1,084** filas (10.0%) → patrón no lineal de anomalías combinadas.
- Consenso entre métodos:
  - Detectadas por los 3 métodos: **502** filas (casos altamente anómalos).
  - Detectadas exactamente por 2 métodos: **731** filas (anómalas consistentes, revisar antes de decidir acción).

**Variables más afectadas (IQR)**
- `Reviews`: **1,925** outliers → distribución extremadamente sesgada; valores muy altos representan apps masivas (probablemente legítimos).
- `Installs Numeric`: **828** outliers → escalas de descargas masivas (1e7–1e9).
- `Price`: **800** outliers → pocos productos de precio elevado (≥ p75 + 1.5·IQR); revisar si son apps premium legítimas.
- `Size`: **564** outliers → tamaños extremos (muy grandes o inusualmente pequeños).
- `Rating`: **504** outliers → incluye valores extremos bajos y el caso inválido (`Rating=19`).

**Interpretación y criterios**
- Muchos outliers provienen de fenómenos de cola larga típicos (popularidad extrema o modelo freemium/premium).
- No se recomienda eliminar masivamente outliers de `Reviews` o `Installs Numeric` sin antes transformar (`log1p`) o agrupar (binning), para no perder información sobre apps exitosas.
- El valor inválido `Rating` debe eliminars. Otros ratings muy bajos pueden mantenerse (aportan contraste) (perfecto si se encuentran entre 1 y 5).
- Outliers en `Price` podrían segmentarse: gratis (0), bajo costo (0 < p ≤ 10), premium (10 < p ≤ 50), ultra premium (>50).


**Conclusión**
El comportamiento extremo de `Reviews` e `Installs Numeric` refleja la naturaleza desigual del mercado (unas pocas apps concentran gran parte de la atención). Un manejo cuidadoso (transformaciones y flags) preservará información útil sin distorsionar el entrenamiento. Se prioriza limpieza puntual (ratings inválidos) sobre eliminación agresiva de outliers.

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

def detect_outliers(df):
    """Detección de outliers usando múltiples métodos"""
    
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Método 1: IQR
    outliers_iqr = pd.DataFrame()
    for col in numeric_df.columns:
        Q1 = numeric_df[col].quantile(0.25)
        Q3 = numeric_df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((numeric_df[col] < Q1 - 1.5 * IQR) | 
                   (numeric_df[col] > Q3 + 1.5 * IQR))
        outliers_iqr[col] = outliers
    
    # Método 2: Z-Score
    from scipy import stats
    z_scores = np.abs(stats.zscore(numeric_df.fillna(numeric_df.median())))
    outliers_zscore = (z_scores > 3)
    
    # Método 3: Isolation Forest
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(numeric_df.fillna(numeric_df.median()))
    iso_forest = IsolationForest(contamination=0.1, random_state=42)
    outliers_iso = iso_forest.fit_predict(scaled_data) == -1
    
    # Visualización
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot 1: Outliers por columna (IQR)
    ax1 = axes[0, 0]
    outlier_counts = outliers_iqr.sum()
    ax1.bar(range(len(outlier_counts)), outlier_counts.values)
    ax1.set_xticks(range(len(outlier_counts)))
    ax1.set_xticklabels(outlier_counts.index, rotation=45, ha='right')
    ax1.set_title('Outliers por Variable (Método IQR)')
    ax1.set_ylabel('Número de Outliers')
    
    # Plot 2: Distribución de outliers por método
    ax2 = axes[0, 1]
    methods_comparison = pd.DataFrame({
        'IQR': outliers_iqr.any(axis=1).sum(),
        'Z-Score': outliers_zscore.any(axis=1).sum(),
        'Isolation Forest': outliers_iso.sum()
    }, index=['Outliers'])
    methods_comparison.T.plot(kind='bar', ax=ax2, legend=False)
    ax2.set_title('Comparación de Métodos de Detección')
    ax2.set_ylabel('Número de Outliers Detectados')
    ax2.set_xlabel('Método')
    
    # Plot 3: Heatmap de outliers
    ax3 = axes[1, 0]
    sample_outliers = outliers_iqr.head(100)
    sns.heatmap(sample_outliers.T, cmap='RdYlBu_r', cbar=False, ax=ax3,
               yticklabels=True, xticklabels=False)
    ax3.set_title('Mapa de Outliers (Primeras 100 filas)')
    ax3.set_xlabel('Observaciones')
    
    # Plot 4: Resumen estadístico
    ax4 = axes[1, 1]
    ax4.axis('off')
    summary_text = f"""
    Resumen de Detección de Anomalías:
    
    • Total de observaciones: {len(df):,}
    • Outliers por IQR: {outliers_iqr.any(axis=1).sum():,} ({outliers_iqr.any(axis=1).sum()/len(df)*100:.1f}%)
    • Outliers por Z-Score: {outliers_zscore.any(axis=1).sum():,} ({outliers_zscore.any(axis=1).sum()/len(df)*100:.1f}%)
    • Outliers por Isolation Forest: {outliers_iso.sum():,} ({outliers_iso.sum()/len(df)*100:.1f}%)
    
    Variables más afectadas:
    {chr(10).join([f'  - {col}: {count:,} outliers' 
                   for col, count in outlier_counts.nlargest(3).items()])}
    
    Investigar outliers antes de eliminar. 
    Pueden contener información valiosa.
    """
    ax4.text(0.1, 0.5, summary_text, transform=ax4.transAxes,
            fontsize=11, verticalalignment='center',
            bbox=dict(boxstyle='round', facecolor='lightyellow'))
    
    plt.suptitle('Análisis de Outliers y Anomalías', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return outliers_iqr, outliers_zscore, outliers_iso

# Ejecutar la detección de outliers en el dataset de aplicaciones
outliers_iqr, outliers_zscore, outliers_iso = detect_outliers(applications_data)

## 2.3. Preparación de los datos

Con base a todos los hallazgos del **Análisis Exploratorio de Datos (EDA)**, aplicaremos las siguientes técnicas de limpieza y transformación:

### 2.3.1. Resumen de problemas detectados

Durante el EDA identificamos:

1. **Duplicados**: 483 filas duplicadas (~4.46%)
2. **Valores imposibles**: Rating = 19 (fuera del rango 1-5)
3. **Valores faltantes**: 
   - Size ≈ 15.6%
   - Rating ≈ 13.6%
   - Current Ver, Android Ver, Content Rating, Type, Price (<1%)
4. **Outliers legítimos**: Distribuciones con colas largas en Reviews, Installs Numeric, Price, Size
5. **Variables con distribuciones sesgadas**: Requieren transformaciones logarítmicas
6. **Inconsistencias**: Necesidad de validar coherencia entre Type y Price

### 2.3.2. Plan de transformación

Aplicaremos las siguientes transformaciones en orden:

1. **Eliminación de duplicados**
2. **Corrección de valores imposibles**
3. **Imputación de valores faltantes** (estrategia por variable)
4. **Validación de consistencia** entre variables relacionadas
5. **Transformaciones de variables numéricas** (log, binning)
6. **Creación de variables derivadas** (features engineering básico)
7. **Resumen final** del dataset limpio


### 2.3.3. Eliminación de Duplicados

Eliminamos las **483 filas duplicadas** detectadas en el EDA para evitar:
- Sesgos en análisis estadísticos
- Sobrepeso de ciertas apps en el modelado
- Distorsión de métricas de evaluación


In [ ]:
# Crear copia del dataset para transformaciones
df_clean = applications_data.copy()

print("=" * 80)
print("PASO 1: ELIMINACIÓN DE DUPLICADOS".center(80))
print("=" * 80)

# Estado inicial
print(f"\nRegistros antes de eliminar duplicados: {len(df_clean):,}")
print(f"Duplicados encontrados: {df_clean.duplicated().sum():,} ({df_clean.duplicated().sum()/len(df_clean)*100:.2f}%)")

# Mostrar algunos ejemplos de duplicados antes de eliminar
if df_clean.duplicated().sum() > 0:
    print("\nEjemplos de aplicaciones duplicadas:")
    duplicated_apps = df_clean[df_clean.duplicated(keep=False)].sort_values('App')
    display(duplicated_apps[['App', 'Category', 'Rating', 'Reviews', 'Installs']].head(10))

# Eliminar duplicados (manteniendo la primera ocurrencia)
df_clean = df_clean.drop_duplicates(keep='first')

# Estado final
print(f"\nRegistros después de eliminar duplicados: {len(df_clean):,}")
print(f"Filas eliminadas: {len(applications_data) - len(df_clean):,}")
print(f"Reducción: {((len(applications_data) - len(df_clean))/len(applications_data)*100):.2f}%")


### 2.3.4. Corrección de Valores Imposibles

Corregimos valores que están fuera del rango válido:
- Cualquier Rating < 1 o > 5 → Eliminar outlier, dado que es imposible que en la escala de Google Play se pueda obtener este tipo de rango, lo que demuestra un verdadero error

#### 2.3.4.1. Análisis de Resultados de Corrección

**Valores imposibles detectados:**

**Ratings inválidos identificados:**
- **1 registro** con Rating = 19.0 (fuera del rango válido 1-5)
- **Aplicación afectada:** Identificada y eliminada del dataset
- **Acción aplicada:** Eliminación completa del registro (no conversión a NaN)

**Verificación de otros valores imposibles:**
- **Reviews**: Sin valores negativos detectados
- **Size**: Sin valores negativos detectados  
- **Price**: Sin valores negativos detectados
- **Installs Numeric**: Sin valores negativos detectados

**Impacto de la corrección:**
- **Registros eliminados:** 1 (impacto mínimo del 0.01%)
- **Integridad del Rating:** 100% de valores dentro del rango [1-5]
- **Registros finales:** 10,357 (después de eliminación de duplicados y valores imposibles)

**Distribución final de Rating (valores válidos):**
- **Rango confirmado:** 1.0 - 5.0 ✓
- **Mediana:** ~4.3 (preservada después de la limpieza)
- **Sin outliers imposibles:** Dataset completamente saneado

**Conclusión:** La corrección fue exitosa eliminando el único valor imposible detectado (Rating=19), garantizando que todos los ratings restantes cumplan con la escala estándar de Google Play Store [1-5].

In [ ]:
print("=" * 80)
print("PASO 2: CORRECCIÓN DE VALORES IMPOSIBLES".center(80))
print("=" * 80)

# Verificar valores de Rating fuera del rango [1, 5]
invalid_ratings = df_clean['Rating'][(df_clean['Rating'] < 1) | (df_clean['Rating'] > 5)]
print(f"\nRatings inválidos encontrados: {len(invalid_ratings)}")

if len(invalid_ratings) > 0:
    print("\nEjemplos de ratings inválidos:")
    invalid_apps = df_clean[df_clean['Rating'].isin(invalid_ratings)]
    display(invalid_apps[['App', 'Category', 'Rating', 'Reviews']].head())
    
    # Mostrar distribución de valores inválidos
    print(f"\nValores inválidos únicos: {sorted(invalid_ratings.dropna().unique())}")
    
    # Eliminar registros con valores inválidos en lugar de poner NaN
    df_clean = df_clean[~((df_clean['Rating'] < 1) | (df_clean['Rating'] > 5))].copy()
    
    print(f"\nRegistros inválidos eliminados.")
    print(f"Total de registros ahora: {len(df_clean)}")
else:
    print("\nNo se encontraron ratings inválidos")

# Verificar otros valores imposibles (negativos en columnas que no pueden serlo)
print("\n" + "-" * 80)
print("Verificando valores negativos en columnas numéricas:")
print("-" * 80)

numeric_cols = ['Reviews', 'Size', 'Price', 'Installs Numeric']
for col in numeric_cols:
    if col in df_clean.columns:
        negative_count = (df_clean[col] < 0).sum()
        if negative_count > 0:
            print(f"{col}: {negative_count} valores negativos encontrados")
            df_clean = df_clean[df_clean[col] >= 0].copy()
            print(f"Registros inválidos eliminados de {col}")
        else:
            print(f"{col}: Sin valores negativos")


### 2.3.5. Validación de Consistencia entre Variables

Validamos y corregimos inconsistencias lógicas entre variables relacionadas:
- **Type vs Price**: Si `Type = 'Free'`, entonces `Price` debe ser 0
- **Type vs Price**: Si `Price > 0`, entonces `Type` debe ser 'Paid'

#### 2.3.5.1. Análisis de Resultados de Validación

**Estado de consistencia encontrado:**

**Consistencia perfecta en apps existentes:**
- **0 apps** marcadas como 'Free' con Price > 0
- **0 apps** marcadas como 'Paid' con Price = 0
- El dataset original ya mantenía la lógica de negocio correcta

**Corrección aplicada:**
- **1 registro** con Type faltante fue corregido automáticamente
- **Estrategia:** Inferencia desde Price (Price = 0 → Type = 'Free')
- **Resultado:** 0 faltantes restantes en Type

**Distribución final validada:**
- **Free**: 9,592 apps (92.6%)
- **Paid**: 765 apps (7.4%)
- **Total**: 10,357 apps con consistencia 100% validada

**Conclusión:** La validación confirmó que el dataset original ya mantenía coherencia lógica entre Type y Price, requiriendo solo la corrección menor de 1 registro con Type faltante.

In [ ]:
print("=" * 80)
print("PASO 3: VALIDACIÓN DE CONSISTENCIA ENTRE VARIABLES".center(80))
print("=" * 80)

# Validar consistencia Type vs Price
print("\nValidando consistencia entre Type y Price:")
print("-" * 80)

# Casos inconsistentes: Type='Free' pero Price > 0
free_but_paid = df_clean[(df_clean['Type'] == 'Free') & (df_clean['Price'] > 0)]
print(f"\nApps marcadas como 'Free' pero con Price > 0: {len(free_but_paid)}")
if len(free_but_paid) > 0:
    display(free_but_paid[['App', 'Category', 'Type', 'Price']].head())
    # Corregir: si Price > 0, cambiar Type a 'Paid'
    df_clean.loc[(df_clean['Type'] == 'Free') & (df_clean['Price'] > 0), 'Type'] = 'Paid'
    print(f"Corregido: Type cambiado a 'Paid'")

# Casos inconsistentes: Type='Paid' pero Price = 0
paid_but_free = df_clean[(df_clean['Type'] == 'Paid') & (df_clean['Price'] == 0)]
print(f"\nApps marcadas como 'Paid' pero con Price = 0: {len(paid_but_free)}")
if len(paid_but_free) > 0:
    display(paid_but_free[['App', 'Category', 'Type', 'Price']].head())
    # Corregir: si Price = 0, cambiar Type a 'Free'
    df_clean.loc[(df_clean['Type'] == 'Paid') & (df_clean['Price'] == 0), 'Type'] = 'Free'
    print(f"Corregido: Type cambiado a 'Free'")

# Inferir Type desde Price cuando Type es NaN
type_missing = df_clean['Type'].isnull()
if type_missing.sum() > 0:
    print(f"\nType faltante en {type_missing.sum()} registros")
    print("Infiriendo Type desde Price...")
    df_clean.loc[type_missing & (df_clean['Price'] == 0), 'Type'] = 'Free'
    df_clean.loc[type_missing & (df_clean['Price'] > 0), 'Type'] = 'Paid'
    remaining_missing = df_clean['Type'].isnull().sum()
    print(f"Type inferido. Faltantes restantes: {remaining_missing}")

print("\nValidación de consistencia completada")
print(f"Distribución final de Type:")
print(df_clean['Type'].value_counts())


### 2.3.6. División Estratificada del Dataset (Train/Val/Test)

**IMPORTANTE: Prevención de Data Leakage**

Antes de realizar cualquier imputación o transformación que calcule estadísticas (medianas, modas, etc.), debemos dividir el dataset para evitar **fugas de información del futuro**.

**Estrategia de división:**
1. Eliminar filas sin Rating (target) → Solo entrenaremos con datos completos
2. División estratificada 70/15/15:
   - **Train (70%)**: Para calcular estadísticas de imputación y ajustar SIN FILTRAR INFORMACIÓN DEL FUTURO
   - **Validation (15%)**: Para validación durante el desarrollo
   - **Test (15%)**: Para evaluación final (nunca se usa hasta el final)
3. Estratificación por Rating para mantener distribución del target (al ser númerico, creamos bins para hacer la estratificación)

**Principio clave:** Las estadísticas (mediana de Size, moda de Content Rating, etc.) se calculan SOLO con train y se aplican a val/test.

**Conclusión:** La gráfica muestra que la distribución de Rating en los tres conjuntos (train, val, test) es prácticamente idéntica, tanto en forma como en valores de media y mediana. Esto indica que la estratificación por bins de Rating fue exitosa y que no hay sesgo entre los splits.
La media y mediana se mantienen estables (≈4.19–4.20 y ≈4.30), lo que garantiza que el modelo se entrenará, validará y evaluará sobre datos representativos y comparables.
Por lo tanto, la división estratificada preserva la estructura original del target y previene data leakage, asegurando resultados robustos y generalizables.

In [ ]:
from sklearn.model_selection import train_test_split

print("=" * 80)
print("PASO 4: DIVISIÓN ESTRATIFICADA DEL DATASET".center(80))
print("=" * 80)

# Estado antes de dividir
print(f"\nDataset después de limpieza básica:")
print(f"  Total de registros: {len(df_clean):,}")
print(f"  Rating faltantes: {df_clean['Rating'].isnull().sum():,} ({df_clean['Rating'].isnull().sum()/len(df_clean)*100:.2f}%)")

# 1. ELIMINAR filas sin Rating (no podemos entrenar con ellas)
df_model = df_clean.dropna(subset=['Rating']).copy()
print(f"\nDespués de eliminar filas sin Rating:")
print(f"  Registros disponibles para modelado: {len(df_model):,}")
print(f"  Registros descartados: {len(df_clean) - len(df_model):,}")

# 2. DIVISIÓN ESTRATIFICADA: Train (70%), Temp (30%)
print("\n" + "-" * 80)
print("Dividiendo dataset (estratificado por Rating)...")
print("-" * 80)

# Crear bins de Rating para estratificación más robusta
df_model['rating_bin'] = pd.cut(df_model['Rating'], bins=[0, 3, 4, 4.5, 5], labels=['Low', 'Medium', 'High', 'VeryHigh'])

train, temp = train_test_split(
    df_model,
    test_size=0.30,
    stratify=df_model['rating_bin'],
    random_state=42
)

# 3. Dividir Temp en Val (50%) y Test (50%) → 15% y 15% del total
val, test = train_test_split(
    temp,
    test_size=0.50,
    stratify=temp['rating_bin'],
    random_state=42
)

# Eliminar columna auxiliar de binning
train = train.drop(columns=['rating_bin'])
val = val.drop(columns=['rating_bin'])
test = test.drop(columns=['rating_bin'])

# Reindexar para evitar problemas
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

print(f"\nDivisión completada:")
print(f"  Train: {len(train):,} registros ({len(train)/len(df_model)*100:.1f}%)")
print(f"  Val:   {len(val):,} registros ({len(val)/len(df_model)*100:.1f}%)")
print(f"  Test:  {len(test):,} registros ({len(test)/len(df_model)*100:.1f}%)")

# 4. VERIFICAR ESTRATIFICACIÓN (distribución de Rating debe ser similar)
print("\n" + "-" * 80)
print("Verificación de estratificación:")
print("-" * 80)

rating_dist = pd.DataFrame({
    'Train_mean': [train['Rating'].mean()],
    'Val_mean': [val['Rating'].mean()],
    'Test_mean': [test['Rating'].mean()],
    'Train_std': [train['Rating'].std()],
    'Val_std': [val['Rating'].std()],
    'Test_std': [test['Rating'].std()]
})
display(rating_dist.round(3))

# Visualización de distribuciones
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (name, data) in zip(axes, [('Train', train), ('Val', val), ('Test', test)]):
    ax.hist(data['Rating'], bins=30, alpha=0.7, edgecolor='black')
    ax.axvline(data['Rating'].mean(), color='red', linestyle='--', label=f'Media: {data["Rating"].mean():.2f}')
    ax.axvline(data['Rating'].median(), color='green', linestyle='--', label=f'Mediana: {data["Rating"].median():.2f}')
    ax.set_title(f'Distribución de Rating - {name}')
    ax.set_xlabel('Rating')
    ax.set_ylabel('Frecuencia')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDivisión estratificada completada exitosamente")
print("IMPORTANTE: A partir de ahora, SOLO usaremos 'train' para calcular estadísticas")

### 2.3.7. Imputación de Valores Faltantes (Sin Data Leakage)

---

####  Calcular estadísticas **solo con `train`**

Todas las estadísticas utilizadas para la imputación (medianas, modas, etc.) se calculan **exclusivamente sobre el conjunto de entrenamiento (`train`)**.  
Esto evita que el modelo tenga acceso indirecto a información de validación o prueba (futuro), previniendo así **data leakage**.

> Ejemplo: las medianas de `Size` se obtienen por combinación `Category × Type` **solo a partir de `train`**, y luego se aplican a `val` y `test`.

---

#### Aplicar las mismas estadísticas a `val` y `test`

Las estadísticas aprendidas del `train` se reutilizan directamente en los conjuntos de validación y prueba.  
No se recalculan en esos conjuntos, lo que asegura **consistencia en la imputación** y **evita fuga de información**.

> Si una categoría no existe en `train`, se aplica una **mediana o moda global** (fallback), calculada también desde `train`.

---

#### Crear *flags* de valores faltantes **antes de imputar**

Antes de reemplazar los valores nulos, se generan columnas binarias (`_missing`) que indican la presencia de datos faltantes.  
Esto permite al modelo aprender si la ausencia de información tiene relevancia predictiva.

> Ejemplo de *flags*:  
> `size_missing`, `content_rating_missing`, `price_missing`, `android_ver_missing`, `current_ver_missing`.

---

#### Métodos de imputación por variable

| Variable | Método de imputación | Nivel de agrupación | Fallback |
|-----------|---------------------|---------------------|-----------|
| **`Size`** | Mediana | `Category × Type` | Mediana global |
| **`Content Rating`** | Moda | `Category` | Moda global |
| **`Android Ver`** | Moda | `Category` | Moda global |
| **`Current Ver`** | Moda | `Category` | Moda global |
| **`Price`** | `0` si `Free`, mediana si `Paid` | `Category` | Mediana global (`Paid`) |

Cada variable mantiene la misma política: **calcular con `train`, aplicar en `val` y `test`**.

---

#### Validación final de imputación

Después del proceso, se confirma que **ningún conjunto tenga valores faltantes**.  
Esto garantiza que la imputación fue exitosa y que los *flags* sean la única señal de ausencia original.

> Ejemplo de salida esperada:
> ```
> Train faltantes: 0
> Val faltantes: 0
> Test faltantes: 0
> ```

---

#### Beneficios de la metodología

| Aspecto | Beneficio |
|----------|------------|
| **Prevención de fuga de datos** | Solo se usan estadísticas del conjunto de entrenamiento. |
| **Consistencia entre conjuntos** | Mismo proceso aplicado a `train`, `val` y `test`. |
| **Trazabilidad de imputaciones** | Los flags `_missing` permiten capturar patrones de ausencia. |
| **Robustez general** | Cada variable se trata según su naturaleza (numérica o categórica). |



In [ ]:
print("=" * 80)
print("PASO 4: IMPUTACIÓN DE VALORES FALTANTES (Sin Data Leakage)".center(80))
print("=" * 80)

print("\nMETODOLOGÍA:")
print("  1. Calcular estadísticas SOLO con train")
print("  2. Aplicar las mismas estadísticas a val y test")
print("  3. Crear flags de faltantes ANTES de imputar")
print("\n" + "=" * 80)

# ==============================================================================
# CREAR FLAGS DE FALTANTES (antes de imputar)
# ==============================================================================
print("\nCreando flags de valores faltantes...")

for df_name, df in [('train', train), ('val', val), ('test', test)]:
    df['size_missing'] = df['Size'].isnull().astype(int)
    df['content_rating_missing'] = df['Content Rating'].isnull().astype(int)
    df['android_ver_missing'] = df['Android Ver'].isnull().astype(int)
    df['current_ver_missing'] = df['Current Ver'].isnull().astype(int)
    df['price_missing'] = df['Price'].isnull().astype(int)

print("Flags creados en train, val y test")

# ==============================================================================
# 1. SIZE: Calcular medianas por Category × Type usando SOLO TRAIN
# ==============================================================================
print("\n" + "-" * 80)
print("Imputando SIZE (mediana por Category × Type)")
print("-" * 80)

# Calcular medianas SOLO con train
size_medians = train.groupby(['Category', 'Type'])['Size'].median()
size_global_median = train['Size'].median()

print(f"Medianas calculadas con train: {len(size_medians)} grupos")
print(f"Mediana global (fallback): {size_global_median:.2f} MB")

# Función para imputar usando mapping precalculado
def impute_size(df, medians_map, global_median):
    df = df.copy()
    for idx, row in df[df['Size'].isnull()].iterrows():
        cat, typ = row['Category'], row['Type']
        if (cat, typ) in medians_map.index:
            df.loc[idx, 'Size'] = medians_map.loc[(cat, typ)]
        else:
            df.loc[idx, 'Size'] = global_median
    return df

# Aplicar a train, val, test
train = impute_size(train, size_medians, size_global_median)
val = impute_size(val, size_medians, size_global_median)
test = impute_size(test, size_medians, size_global_median)

print(f"Size imputado en todos los conjuntos")
print(f"   Train faltantes: {train['Size'].isnull().sum()}")
print(f"   Val faltantes: {val['Size'].isnull().sum()}")
print(f"   Test faltantes: {test['Size'].isnull().sum()}")

# ==============================================================================
# 2. CONTENT RATING: Moda por Category (solo train)
# ==============================================================================
print("\n" + "-" * 80)
print("Imputando CONTENT RATING (moda por Category)")
print("-" * 80)

content_rating_modes = train.groupby('Category')['Content Rating'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
content_rating_global_mode = train['Content Rating'].mode()[0]

print(f"Modas calculadas con train: {len(content_rating_modes)} categorías")

def impute_content_rating(df, modes_map, global_mode):
    df = df.copy()
    for idx, row in df[df['Content Rating'].isnull()].iterrows():
        cat = row['Category']
        if cat in modes_map.index and pd.notna(modes_map.loc[cat]):
            df.loc[idx, 'Content Rating'] = modes_map.loc[cat]
        else:
            df.loc[idx, 'Content Rating'] = global_mode
    return df

train = impute_content_rating(train, content_rating_modes, content_rating_global_mode)
val = impute_content_rating(val, content_rating_modes, content_rating_global_mode)
test = impute_content_rating(test, content_rating_modes, content_rating_global_mode)

print(f"Content Rating imputado")

# ==============================================================================
# 3. ANDROID VER: Moda por Category (solo train)
# ==============================================================================
print("\n" + "-" * 80)
print("Imputando ANDROID VER (moda por Category)")
print("-" * 80)

android_ver_modes = train.groupby('Category')['Android Ver'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
android_ver_global_mode = train['Android Ver'].mode()[0]

def impute_android_ver(df, modes_map, global_mode):
    df = df.copy()
    for idx, row in df[df['Android Ver'].isnull()].iterrows():
        cat = row['Category']
        if cat in modes_map.index and pd.notna(modes_map.loc[cat]):
            df.loc[idx, 'Android Ver'] = modes_map.loc[cat]
        else:
            df.loc[idx, 'Android Ver'] = global_mode
    return df

train = impute_android_ver(train, android_ver_modes, android_ver_global_mode)
val = impute_android_ver(val, android_ver_modes, android_ver_global_mode)
test = impute_android_ver(test, android_ver_modes, android_ver_global_mode)

print(f"Android Ver imputado")

# ==============================================================================
# 4. CURRENT VER: Moda por Category (solo train)
# ==============================================================================
print("\n" + "-" * 80)
print("Imputando CURRENT VER (moda por Category)")
print("-" * 80)

current_ver_modes = train.groupby('Category')['Current Ver'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
current_ver_global_mode = train['Current Ver'].mode()[0]

def impute_current_ver(df, modes_map, global_mode):
    df = df.copy()
    for idx, row in df[df['Current Ver'].isnull()].iterrows():
        cat = row['Category']
        if cat in modes_map.index and pd.notna(modes_map.loc[cat]):
            df.loc[idx, 'Current Ver'] = modes_map.loc[cat]
        else:
            df.loc[idx, 'Current Ver'] = global_mode
    return df

train = impute_current_ver(train, current_ver_modes, current_ver_global_mode)
val = impute_current_ver(val, current_ver_modes, current_ver_global_mode)
test = impute_current_ver(test, current_ver_modes, current_ver_global_mode)

print(f"Current Ver imputado")

# ==============================================================================
# 5. PRICE: 0 si Free, mediana por Category si Paid (solo train)
# ==============================================================================
print("\n" + "-" * 80)
print("Imputando PRICE (0 si Free, mediana por Category si Paid)")
print("-" * 80)

# Calcular medianas de Price para apps Paid por Category (solo train)
price_medians_paid = train[train['Type'] == 'Paid'].groupby('Category')['Price'].median()
price_global_median_paid = train[train['Type'] == 'Paid']['Price'].median()

def impute_price(df, medians_paid_map, global_median_paid):
    df = df.copy()
    # Free apps → 0
    mask_free = (df['Type'] == 'Free') & df['Price'].isnull()
    df.loc[mask_free, 'Price'] = 0
    
    # Paid apps → mediana por Category
    for idx, row in df[(df['Type'] == 'Paid') & df['Price'].isnull()].iterrows():
        cat = row['Category']
        if cat in medians_paid_map.index and pd.notna(medians_paid_map.loc[cat]):
            df.loc[idx, 'Price'] = medians_paid_map.loc[cat]
        else:
            df.loc[idx, 'Price'] = global_median_paid
    return df

train = impute_price(train, price_medians_paid, price_global_median_paid)
val = impute_price(val, price_medians_paid, price_global_median_paid)
test = impute_price(test, price_medians_paid, price_global_median_paid)

print(f"Price imputado")

# ==============================================================================
# RESUMEN FINAL
# ==============================================================================
print("\n" + "=" * 80)
print("IMPUTACIÓN COMPLETADA SIN DATA LEAKAGE")
print("=" * 80)

print("\nValores faltantes restantes:")
for name, df in [('Train', train), ('Val', val), ('Test', test)]:
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if len(missing) == 0:
        print(f"  {name}: Sin valores faltantes")
    else:
        print(f"  {name}: {missing.to_dict()}")

print("\nFlags de trazabilidad creados:")
print(f"  - size_missing")
print(f"  - content_rating_missing")
print(f"  - android_ver_missing")
print(f"  - current_ver_missing")
print(f"  - price_missing")

### 2.3.7. Transformaciones de Variables Numéricas (Sin Data Leakage)

En esta etapa se aplicaron transformaciones diseñadas para **reducir la asimetría**, **mejorar la interpretabilidad** y **preparar las variables numéricas** para los modelos, garantizando que ninguna transformación utilizara información de validación o prueba.

---

#### 2.3.7.1. Log-transformaciones

Se aplicaron transformaciones logarítmicas a variables con distribuciones altamente sesgadas y colas largas.  
Estas transformaciones se realizaron **de forma directa (sin estadísticos derivados)**, por lo que **no generan data leakage**.

**Variables transformadas:**
- `Reviews_log = log1p(Reviews)`
- `Installs_log = log1p(Installs Numeric)`
- `Size_log = log1p(Size)`

**Resultados en el conjunto de entrenamiento:**
- `Reviews_log`: Media = **8.26**, Mediana = **8.44**  
- `Installs_log`: Media = **12.18**, Mediana = **13.12**  
- `Size_log`: Media = **2.64**, Mediana = **2.64**

> Estas transformaciones suavizan la escala exponencial de las variables y mejoran la linealidad con respecto al target.


---

#### 2.3.7.3. Variables binarias derivadas

Se generaron nuevas variables booleanas basadas en reglas de negocio fijas, lo que facilita que el modelo capture relaciones no lineales simples.

**Variables creadas:**
- `is_free`: 1 si la app es gratuita (`Type == 'Free'`) → **5798 apps (93.2%)**
- `is_large_app`: 1 si el tamaño > 50 MB  
- `has_high_installs`: 1 si `Installs Numeric > 1,000,000` → **1793 apps (28.8%)**
- `is_top_category`: 1 si pertenece a `FAMILY` o `GAME`
- `is_everyone_rated`: 1 si `Content Rating == 'Everyone'`
- `large_and_popular`: combinación de `is_large_app` & `has_high_installs`

> Estas variables mejoran la capacidad del modelo para identificar patrones de negocio relevantes (por ejemplo, apps grandes y populares tienden a obtener mejores ratings).



In [ ]:
print("=" * 80)
print("PASO 5: TRANSFORMACIONES DE VARIABLES NUMERICAS (Sin Data Leakage)".center(80))
print("=" * 80)

# ==============================================================================
# 1. LOG-TRANSFORMACIONES (sin data leakage, son transformaciones puntuales)
# ==============================================================================
print("\nAplicando transformaciones logarítmicas...")
print("-" * 80)

for df_name, df in [('train', train), ('val', val), ('test', test)]:
    df['Reviews_log'] = np.log1p(df['Reviews'])
    df['Installs_log'] = np.log1p(df['Installs Numeric'])
    df['Size_log'] = np.log1p(df['Size'])

print("Variables originales eliminadas tras log-transformación: Reviews, Installs Numeric, Size")
print("Transformaciones log aplicadas a train, val, test")
print(f"\n   Train - Reviews_log: Media {train['Reviews_log'].mean():.2f}, Mediana {train['Reviews_log'].median():.2f}")
print(f"   Train - Installs_log: Media {train['Installs_log'].mean():.2f}, Mediana {train['Installs_log'].median():.2f}")
print(f"   Train - Size_log: Media {train['Size_log'].mean():.2f}, Mediana {train['Size_log'].median():.2f}")


# ==============================================================================
# 2. VARIABLES BINARIAS (sin data leakage, son reglas fijas)
# ==============================================================================
print("\n\nCreando variables binarias...")
print("-" * 80)

for df_name, df in [('train', train), ('val', val), ('test', test)]:
    df['is_free'] = (df['Type'] == 'Free').astype(int)
    df['is_large_app'] = (df['Size'] > 50).astype(int)
    df['has_high_installs'] = (df['Installs Numeric'] > 1000000).astype(int)
    df['is_top_category'] = df['Category'].isin(['FAMILY', 'GAME']).astype(int)
    df['is_everyone_rated'] = (df['Content Rating'] == 'Everyone').astype(int)
    df['large_and_popular'] = (df['is_large_app'] & df['has_high_installs']).astype(int)

print("Variables binarias creadas")
print(f"\n   Train - is_free: {train['is_free'].sum()} ({train['is_free'].mean()*100:.1f}%)")
print(f"   Train - has_high_installs: {train['has_high_installs'].sum()} ({train['has_high_installs'].mean()*100:.1f}%)")

print("\nTransformaciones completadas sin data leakage")

### 2.3.8. Creacion de Variables Derivadas (Feature Engineering Básico)

Creamos nuevas variables combinando informacion existente para capturar patrones mas complejos, y eliminamos aquellas redundantes (procesos intermedios, reemplazadas por log1p, etc), que no nos brindan señal predictiva:


In [ ]:
print("=" * 80)
print("PASO 6: FEATURE ENGINEERING (Sin Data Leakage)".center(80))
print("=" * 80)

for df_name, df in [('train', train), ('val', val), ('test', test)]:
    df['review_rate'] = df['Reviews'] / (df['Installs Numeric'] + 1)
    df['Last Updated Parsed'] = pd.to_datetime(df['Last Updated'], errors='coerce')
    reference_date = pd.to_datetime('2025-10-02')
    df['days_since_update'] = (reference_date - df['Last Updated Parsed']).dt.days
    df['update_recency'] = pd.cut(
        df['days_since_update'],
        bins=[-1, 30, 90, 180, 365, 730, 10000],
        labels=['<1 month', '1-3 months', '3-6 months', '6-12 months', '1-2 years', '>2 years']
    )
    df['size_per_install'] = df['Size'] / (df['Installs Numeric'] + 1)
    

print("\nCalculando popularity score (normalizadores desde train)...")
installs_min_train = train['Installs Numeric'].min()
installs_max_train = train['Installs Numeric'].max()
reviews_min_train = train['Reviews'].min()
reviews_max_train = train['Reviews'].max()

for df_name, df in [('train', train), ('val', val), ('test', test)]:
    installs_norm = (df['Installs Numeric'] - installs_min_train) / (installs_max_train - installs_min_train)
    reviews_norm = (df['Reviews'] - reviews_min_train) / (reviews_max_train - reviews_min_train)
    df['popularity_score'] = (installs_norm * 0.7 + reviews_norm * 0.3) * 100

print("Features derivadas creadas para obtener señal predictiva, sin data leakage")

print("------------------------")


print("Eliminación de columnas redudantes")

redundant_cols = [
    'Reviews', 'Installs Numeric', 'Size', 'Installs', 'Genres', 'Last Updated', 'Last Updated Parsed',
    'Current Ver', 'Android Ver', 'days_since_update'
]


for df in [train, val, test]:
    for col in redundant_cols:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)
print("Columnas redundantes eliminadas antes del encoding.")


### 2.3.9 Manejo de categorías raras en la variable `Category`

En esta sección se agrupan las categorías poco frecuentes dentro de la variable `Category`.  
El objetivo es reducir el impacto del desbalance categórico y evitar que categorías con muy pocos registros afecten el aprendizaje del modelo.

Para ello, se define un **umbral (`threshold = 70`)** sobre el conjunto de entrenamiento.  
Todas las categorías con frecuencia menor a 70 se agrupan en una nueva categoría denominada **"Other"**.  
Posteriormente, se aplica el mismo mapeo al conjunto de validación y prueba, garantizando coherencia entre los tres splits y evitando fuga de información.


In [ ]:
threshold = 70
freq_train = train['Category'].value_counts()
main_cats = freq_train[freq_train >= threshold].index.tolist()


def map_to_other(df, col, keep):
    df = df.copy()
    df[col] = df[col].apply(lambda x: x if x in keep else 'Other')
    return df

train = map_to_other(train, 'Category', set(main_cats))
val   = map_to_other(val,   'Category', set(main_cats))
test  = map_to_other(test,  'Category', set(main_cats))

print(f"Categorías 'Other' en train: {train['Category'].value_counts(normalize=True)['Other']}")
print(f"Categorías 'Other' en validation: {val['Category'].value_counts(normalize=True)['Other']}")
print(f"Categorías 'Other' en test: {test['Category'].value_counts(normalize=True)['Other']}")

### 2.3.10 Manejo de categorías raras en la variable `Content Rating`

En esta sección se analizan las categorías presentes en la variable `Content Rating`, con el objetivo de identificar valores poco representativos que podrían introducir ruido en el modelo.

Durante la exploración se observaron las siguientes clases principales:
- **Everyone**
- **Teen**
- **Mature 17+**
- **Everyone 10+**

Y dos categorías extremadamente raras:
- **Adults only 18+** (2 registros)
- **Unrated** (1 registro)

Dado que estas últimas representan menos del **0.05 %** del total de observaciones, se decidió **eliminarlas directamente** en lugar de agruparlas bajo una categoría "Other".  
Esta decisión se justifica porque su frecuencia es demasiado baja para aportar señal estadística y no se espera que aparezcan con relevancia en datos futuros.

In [ ]:
train = train[~train['Content Rating'].isin(['Adults only 18+', 'Unrated'])].copy()
val   = val[~val['Content Rating'].isin(['Adults only 18+', 'Unrated'])].copy()
test  = test[~test['Content Rating'].isin(['Adults only 18+', 'Unrated'])].copy()

print(f"Content Rating en train: {train['Content Rating'].value_counts()}")
print("-----")
print(f"Content Rating en validation: {val['Content Rating'].value_counts()}")
print("-----")
print(f"Content Rating en test: {test['Content Rating'].value_counts()}")


### 2.3.11 Resumen final del dataset dividido (Train / Validation / Test)

En esta sección se presenta un **resumen general** del estado final de los datos tras todo el proceso de limpieza, depuración y división en conjuntos de entrenamiento, validación y prueba.  
El objetivo es validar que las transformaciones previas (eliminación de duplicados, tratamiento de valores faltantes, corrección de outliers y manejo de variables categóricas) se hayan aplicado correctamente y que las tres particiones mantengan coherencia estructural y estadística.



In [ ]:
# ==============================================
# 2.3.11 - Resumen final del dataset dividido
# ORIGINAL vs (TRAIN / VALIDATION / TEST)
# ==============================================

import pandas as pd
from IPython.display import display

print("=" * 80)
print("RESUMEN FINAL: DATASET DIVIDIDO (Train / Validation / Test)".center(80))
print("=" * 80)

print("\n" + "=" * 40)
print("COMPARACIÓN: ORIGINAL vs SPLITS")
print("=" * 40)

# ---------- Función de resumen por dataset ----------
def get_summary(df, name):
    return [
        name,
        f"{len(df):,}",
        f"{len(df.columns)}",
        f"{df.duplicated().sum():,}",
        f"{df['Rating'].isnull().sum():,}" if 'Rating' in df.columns else "N/A",
        f"{df['Size'].isnull().sum():,}" if 'Size' in df.columns else "N/A",
        f"{df['Price'].isnull().sum():,}" if 'Price' in df.columns else "N/A",
        f"{df['Type'].isnull().sum():,}" if 'Type' in df.columns else "N/A",
        f"{((df['Rating'] > 5) | (df['Rating'] < 1)).sum():,}" if 'Rating' in df.columns else "N/A",
        f"{df.memory_usage(deep=True).sum() / 1024**2:.2f}"
    ]

# ---------- Construcción de la tabla comparativa ----------
summary_rows = [
    get_summary(applications_data, "Original"),
    get_summary(train, "Train"),
    get_summary(val, "Validation"),
    get_summary(test, "Test")
]

comparison = pd.DataFrame(
    summary_rows,
    columns=[
        "Conjunto",
        "Total de registros",
        "Total de columnas",
        "Duplicados",
        "Rating faltantes",
        "Size faltantes",
        "Price faltantes",
        "Type faltantes",
        "Ratings inválidos (>5 o <1)",
        "Memoria (MB)"
    ]
)

display(comparison)

# ---------- Listado de features por split ----------
print("\n" + "=" * 40)
print("LISTADO DE FEATURES EN CADA CONJUNTO")
print("=" * 40)
for name, df in [('Train', train), ('Validation', val), ('Test', test)]:
    print(f"\n{name}: Total de features = {len(df.columns)}")
    print("columnas:")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:3d}. {col}")
    if len(df.columns) > 30:
        print("  ...")
    print("Últimas 5 columnas:")
    for i, col in enumerate(df.columns[-5:], len(df.columns)-4):
        print(f"  {i:3d}. {col}")

# ---------- Estadísticas descriptivas (numéricas) ----------
print("\n" + "=" * 40)
print("ESTADÍSTICAS DESCRIPTIVAS (Variables numéricas - Train)")
print("=" * 40)

# Variables numéricas más relevantes para el análisis
key_numeric = [
    'Rating', 
    'Price', 
    'review_rate', 
    'size_per_install', 
    'popularity_score',
    'Reviews_log',
    'Installs_log',
    'Size_log'
]

# Filtrar solo las columnas que existen realmente en el dataset
available_numeric = [col for col in key_numeric if col in train.columns]

# Mostrar la descripción
if available_numeric:
    print(f"\nVariables incluidas en el análisis: {', '.join(available_numeric)}")
    display(train[available_numeric].describe().round(3).T)
else:
    print("No se encontraron las columnas seleccionadas en el conjunto de entrenamiento.")

# ---------- Mensaje final ----------
print("\n" + "=" * 80)
print("TRANSFORMACIÓN Y DIVISIÓN COMPLETADAS EXITOSAMENTE")
print("=" * 80)
print(f"\nConjuntos disponibles: train, val, test")
print(f"Dimensiones finales: Train={train.shape[0]:,}, Val={val.shape[0]:,}, Test={test.shape[0]:,}")
print(f"Memoria utilizada (train): {train.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


### 2.3.12 Análisis de relevancia de variables mediante Información Mutua

En esta etapa se aplicó la métrica de **Información Mutua** para cuantificar la dependencia estadística entre cada variable numérica y la variable objetivo `Rating`.  
Este análisis permite identificar qué variables aportan mayor información al modelo, sin asumir relaciones lineales, ayudando a seleccionar los predictores más relevantes antes del entrenamiento.

#### 2.3.12.1. Interpretación de resultados

La gráfica muestra la **importancia relativa de cada variable** según su grado de información compartida con `Rating`.  
Se observa que:

- **`popularity_score`** es la variable más influyente, con el valor de información mutua más alto (~0.34). Esto indica una fuerte relación entre la popularidad de la app (descargas y reseñas combinadas) y su calificación promedio.  
- **`Reviews_log`** y **`review_rate`** también presentan una asociación significativa con el rating, lo que refuerza la idea de que la participación y satisfacción de los usuarios están estrechamente ligadas a la puntuación final.  
- Variables como **`Installs_log`**, **`size_per_install`** y **`has_high_installs`** muestran una contribución media, aportando información adicional relacionada con la escala de uso y la eficiencia de la aplicación.  
- En contraste, variables como **`Price`**, **`price_missing`**, **`is_top_category`** o **`current_ver_missing`** tienen una influencia muy baja, lo que sugiere que su aporte al modelo sería marginal.


In [ ]:
from sklearn.feature_selection import mutual_info_regression
import matplotlib.pyplot as plt

# Selección de variables predictoras: solo numéricas (int, float)
target = 'Rating'
ignore_cols = [target]
X_cols = [
    col for col in train.columns
    if col not in ignore_cols
    and pd.api.types.is_numeric_dtype(train[col])
]

print(X_cols)

X = train[X_cols].copy()
y = train[target]

# Calcular información mutua
mi_scores = mutual_info_regression(X, y, random_state=42)
mi_df = pd.DataFrame({'Variable': X_cols, 'MI_Score': mi_scores}).sort_values('MI_Score', ascending=False)

# Visualización
plt.figure(figsize=(10, 6))
plt.barh(mi_df['Variable'], mi_df['MI_Score'], color='teal')
plt.xlabel('Información Mutua con Rating')
plt.title('Importancia de Variables (Información Mutua)')
plt.gca().invert_yaxis()
plt.show()

# Mostrar tabla ordenada
display(mi_df)

# Eliminar variables con MI muy baja (<0.01)
low_mi_vars = mi_df[mi_df['MI_Score'] < 0.01]['Variable'].tolist()
print(f"\nVariables con baja información mutua (<0.01) que pueden eliminarse inicialmente:")
for v in low_mi_vars:
    print(f"  - {v}")

# Variables relevantes para modelado
selected_vars = mi_df[mi_df['MI_Score'] >= 0.01]['Variable'].tolist()
print(f"\nVariables seleccionadas para modelado inicial ({len(selected_vars)}):")
print(selected_vars)

### 2.3.13 Verificación de correlación y análisis de multicolinealidad

En esta etapa se evaluó la **correlación lineal entre las variables seleccionadas** tras el análisis de información mutua, con el fin de detectar posibles casos de **multicolinealidad**.  
La multicolinealidad ocurre cuando dos o más variables están fuertemente correlacionadas entre sí, lo que puede distorsionar la interpretación de los modelos y afectar la estabilidad de los coeficientes en algoritmos lineales (por ejemplo, regresión o modelos basados en pesos).

Para ello, se calculó la **matriz de correlación de Pearson** considerando únicamente las variables numéricas seleccionadas y se visualizó mediante un mapa de calor.

#### 2.3.13.1. Interpretación de la matriz

- Se observa una **alta correlación entre `Reviews_log` e `Installs_log` (r ≈ 0.96)**, lo que indica que ambas variables transmiten información muy similar: las aplicaciones con muchas reseñas suelen tener también un gran número de instalaciones.  
  Por tanto, mantener ambas podría ser redundante en modelos sensibles a multicolinealidad.  
- También existe una correlación considerable entre **`has_high_installs`** y las variables anteriores (`Installs_log` y `Reviews_log`), dado que esta variable binaria deriva del mismo concepto (nivel alto de descargas).  
- El resto de las variables presentan correlaciones moderadas o bajas, lo cual es positivo: **no se evidencia colinealidad severa** fuera del grupo relacionado con las métricas de descargas y reseñas.  
- Variables como `review_rate`, `size_per_install`, `popularity_score`, `Size_log` y `large_and_popular` muestran relaciones débiles o independientes entre sí, lo que las hace adecuadas para conservarlas.

#### 2.3.13.2. Decisión
Se toma la decisión de eliminar Installs_log dado que Reviews_Log explica su comportamiento.

In [ ]:
# Análisis de multicolinealidad entre variables seleccionadas por información mutua
def multicollinearity_analysis(df, selected_vars):
    """
    Analiza la multicolinealidad entre las variables seleccionadas (sin incluir el target).
    """
    # Solo variables numéricas seleccionadas (sin el target)
    selected_numeric = [v for v in selected_vars if pd.api.types.is_numeric_dtype(df[v])]
    corr_matrix = df[selected_numeric].corr(method='pearson')

    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, vmin=-1, vmax=1)
    plt.title('Matriz de Correlación (Pearson) - Multicolinealidad entre variables seleccionadas')
    plt.tight_layout()
    plt.show()

    # Mostrar pares con alta correlación (|corr| > 0.8)
    print("\nPares de variables con posible multicolinealidad (|corr| > 0.8):")
    high_corr = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    for col in high_corr.columns:
        for idx in high_corr.index:
            corr_val = high_corr.loc[idx, col]
            if abs(corr_val) > 0.8:
                print(f"  • {idx} y {col}: {corr_val:+.2f}")

# Ejecutar el análisis de multicolinealidad solo para las variables seleccionadas
multicollinearity_analysis(train, selected_vars)

In [ ]:
for df in [train, val, test]:
    if 'Installs_log' in df.columns:
        df.drop(columns=['Installs_log'], inplace=True)
print("Columna 'Installs_log' eliminada por multicolinealidad con 'Reviews_log'.")

### 2.3.14 Codificación de variables categóricas (One-Hot Encoding)

En esta fase se transformaron las variables categóricas del conjunto de datos en formato numérico mediante **One-Hot Encoding (OHE)**.  
Este proceso es esencial para los modelos de machine learning que requieren valores numéricos de entrada, permitiendo representar cada categoría como una columna binaria independiente.

#### 2.3.14.1. Proceso aplicado

1. Se identificaron todas las variables categóricas (`object` o `category`) excepto el identificador `App`.  
2. Se aplicó **One-Hot Encoding** sobre estas variables, creando una columna por cada categoría posible.  
3. Para garantizar consistencia entre los conjuntos `train`, `validation` y `test`, se implementó una función que:
   - Asegura que **todos los conjuntos contengan las mismas columnas codificadas**.  
   - Agrega columnas faltantes con valores `0` en caso de que una categoría no esté presente en un conjunto específico.  
   - Elimina cualquier columna extra que no esté en la estructura original de `train`.  
4. Finalmente, los tres conjuntos (`train`, `val`, `test`) quedaron alineados en número y orden de columnas, listos para el modelado.

#### 2.3.14.2. Resultado

Tras la codificación:
- Se convirtieron correctamente todas las variables categóricas a formato numérico.  
- El número de columnas aumentó, reflejando las nuevas variables dummy generadas por OHE.  
- Se verificó que la estructura final es idéntica en los tres conjuntos:  

| Conjunto | Total de columnas |
|-----------|------------------|
| **Train** | 57 |
| **Validation** | igual a Train |
| **Test** | igual a Train |


In [ ]:
cat_cols = [
    c for c in train.columns
    if (train[c].dtype == 'object' or str(train[c].dtype) == 'category')
    and c not in ['App']
]

print(cat_cols)
train_dummies = pd.get_dummies(train, columns=cat_cols, drop_first=False)
dummy_cols = [col for col in train_dummies.columns if col not in train.columns or col in cat_cols]

def align_dummies(df, cat_cols, dummy_cols):
    df_dummies = pd.get_dummies(df, columns=cat_cols, drop_first=False)
    for col in dummy_cols:
        if col not in df_dummies.columns:
            df_dummies[col] = 0
    extra_cols = set(df_dummies.columns) - set(train_dummies.columns)
    df_dummies = df_dummies.drop(columns=list(extra_cols))
    df_dummies = df_dummies[train_dummies.columns]
    return df_dummies

val_dummies = align_dummies(val, cat_cols, dummy_cols)
test_dummies = align_dummies(test, cat_cols, dummy_cols)

train = train_dummies.copy()
val = val_dummies.copy()
test = test_dummies.copy()

print(f"Variables categóricas codificadas y alineadas: Train={train.shape[1]}, Val={val.shape[1]}, Test={test.shape[1]}")


# 3. Modelos Avanzados de ML

En esta sección implementaremos múltiples familias de algoritmos de machine learning para predecir el rating de aplicaciones:

1. **Support Vector Machines (SVM)**: Modelos lineales y no lineales con diferentes kernels
2. **Modelos basados en Árboles**: Decision Trees, Random Forest, Extra Trees
3. **Métodos de Ensamble**: Gradient Boosting, XGBoost, LightGBM
4. **Selección de Características**: Técnicas para identificar las variables más relevantes
5. **Análisis de Umbrales**: Conversión a clasificación binaria y optimización

El objetivo es comparar el rendimiento de diferentes algoritmos y seleccionar el mejor modelo para producción.

In [ ]:
# Importar librerías necesarias para modelos avanzados de ML
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, RFE
from sklearn.decomposition import PCA

# XGBoost y LightGBM
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Utilidades
import time
import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas exitosamente")

In [ ]:
# Definir constantes globales
RANDOM_STATE = 42
CV_FOLDS = 3
THRESHOLD = 4.3  # Umbral para clasificación binaria (rating alto/bajo)
PCA_VARIANCE_THRESHOLD = 0.95
CORRELATION_THRESHOLD = 0.95
N_JOBS = -1  # Usar todos los cores disponibles

# Configurar seeds para reproducibilidad
np.random.seed(RANDOM_STATE)
import random
random.seed(RANDOM_STATE)

print(f"Constantes definidas:")
print(f"  - RANDOM_STATE: {RANDOM_STATE}")
print(f"  - CV_FOLDS: {CV_FOLDS}")
print(f"  - THRESHOLD: {THRESHOLD}")
print(f"  - N_JOBS: {N_JOBS}")

In [ ]:
# Inicializar diccionario para almacenar resultados de todos los modelos
model_results = {}

print("Diccionario model_results inicializado")
print("Estructura: model_results['Model_Name'] = {'mae', 'rmse', 'r2', 'train_time', ...}")

In [ ]:
# Validar que existan las variables necesarias del notebook
print("=" * 80)
print("VALIDACIÓN DE DATASETS".center(80))
print("=" * 80)

# Verificar que existan los conjuntos de datos
required_vars = ['train', 'val', 'test']
for var_name in required_vars:
    if var_name not in locals() and var_name not in globals():
        raise ValueError(f"Variable '{var_name}' no encontrada. Ejecutar celdas anteriores.")

print(f"\nConjuntos de datos encontrados:")
print(f"  - Train: {train.shape}")
print(f"  - Validation: {val.shape}")
print(f"  - Test: {test.shape}")

# Separar features (X) y target (y)
target_col = 'Rating'
feature_cols = [col for col in train.columns if col not in [target_col, 'App']]

X_train = train[feature_cols].copy()
y_train = train[target_col].copy()

X_val = val[feature_cols].copy()
y_val = val[target_col].copy()

X_test = test[feature_cols].copy()
y_test = test[target_col].copy()

print(f"\nFeatures (X):")
print(f"  - X_train: {X_train.shape}")
print(f"  - X_val: {X_val.shape}")
print(f"  - X_test: {X_test.shape}")

print(f"\nTarget (y):")
print(f"  - y_train: {y_train.shape} | Rango: [{y_train.min():.2f}, {y_train.max():.2f}]")
print(f"  - y_val: {y_val.shape} | Rango: [{y_val.min():.2f}, {y_val.max():.2f}]")
print(f"  - y_test: {y_test.shape} | Rango: [{y_test.min():.2f}, {y_test.max():.2f}]")

# Validaciones de integridad
assert X_train.shape[1] == X_val.shape[1] == X_test.shape[1], "Dimensiones de features no coinciden"
assert not X_train.isnull().any().any(), "X_train contiene valores NaN"
assert not X_val.isnull().any().any(), "X_val contiene valores NaN"
assert not X_test.isnull().any().any(), "X_test contiene valores NaN"
assert y_train.min() >= 1 and y_train.max() <= 5, "Valores de y_train fuera del rango [1,5]"

print("\n✓ Validación completada exitosamente")
print(f"\nTotal de features disponibles: {len(feature_cols)}")
print(f"Primeras 10 features: {feature_cols[:10]}")

## 3.1. Support Vector Machines (SVM)

### ¿Qué es SVM?

**Support Vector Machines (SVM)** es un algoritmo de aprendizaje supervisado que puede usarse tanto para clasificación como para regresión (SVR - Support Vector Regression). En el caso de regresión, SVM busca encontrar un hiperplano que mejor se ajuste a los datos, maximizando el margen de tolerancia (epsilon) alrededor de las predicciones.

### Características principales:

1. **Kernels**: SVM puede usar diferentes funciones kernel para capturar relaciones no lineales:
   - **Linear**: Para relaciones lineales simples
   - **RBF (Radial Basis Function)**: Para relaciones no lineales complejas (el más común)
   - **Polynomial**: Para relaciones polinómicas

2. **Hiperparámetros clave**:
   - **C**: Controla el trade-off entre margen suave y error de entrenamiento (regularización)
   - **gamma**: Define la influencia de un solo ejemplo de entrenamiento (solo para kernels RBF y poly)
   - **epsilon**: Ancho del tubo de tolerancia en SVR

### ¿Por qué SVM necesita normalización?

**SVM es extremadamente sensible a la escala de las características** por las siguientes razones:

1. **Cálculo de distancias**: SVM se basa en calcular distancias entre puntos en el espacio de características. Si una variable tiene un rango de [0, 1] y otra de [0, 1,000,000], la segunda dominará el cálculo de distancias.

2. **Optimización del hiperplano**: El algoritmo busca maximizar el margen, que depende de las distancias. Sin normalización, las features con mayor escala tendrán un impacto desproporcionado.

3. **Convergencia**: La optimización numérica converge mucho más rápido cuando todas las features están en la misma escala.

4. **Interpretación de gamma**: El parámetro gamma controla la influencia de cada punto. Si las features tienen escalas diferentes, gamma afectará de manera desigual a cada dimensión.

### Estrategia de normalización:

Usaremos **StandardScaler** (estandarización) que transforma cada feature para tener:
- Media = 0
- Desviación estándar = 1

Fórmula: `z = (x - μ) / σ`

**Importante**: El scaler se ajusta SOLO con los datos de entrenamiento y luego se aplica a validación y test para evitar data leakage.


In [ ]:
# Task 2.2: Implementar preprocesamiento con StandardScaler
print("=" * 80)
print("PREPROCESAMIENTO: STANDARDSCALER PARA SVM".center(80))
print("=" * 80)

# Crear StandardScaler
scaler_svm = StandardScaler()

# Ajustar con X_train (IMPORTANTE: solo con train para evitar data leakage)
scaler_svm.fit(X_train)

# Transformar X_train, X_val, X_test
X_train_scaled = scaler_svm.transform(X_train)
X_val_scaled = scaler_svm.transform(X_val)
X_test_scaled = scaler_svm.transform(X_test)

print(f"\nDatos escalados:")
print(f"  - X_train_scaled: {X_train_scaled.shape}")
print(f"  - X_val_scaled: {X_val_scaled.shape}")
print(f"  - X_test_scaled: {X_test_scaled.shape}")

# Validar que no haya NaN después del scaling
assert not np.isnan(X_train_scaled).any(), "X_train_scaled contiene NaN"
assert not np.isnan(X_val_scaled).any(), "X_val_scaled contiene NaN"
assert not np.isnan(X_test_scaled).any(), "X_test_scaled contiene NaN"

print("\n✓ Validación: No hay valores NaN después del scaling")

# Mostrar estadísticas de los datos escalados
print(f"\nEstadísticas de X_train_scaled:")
print(f"  - Media: {X_train_scaled.mean():.6f} (debería estar cerca de 0)")
print(f"  - Desviación estándar: {X_train_scaled.std():.6f} (debería estar cerca de 1)")
print(f"  - Rango: [{X_train_scaled.min():.2f}, {X_train_scaled.max():.2f}]")


In [ ]:
# Task 2.3: Realizar análisis PCA
print("=" * 80)
print("ANÁLISIS PCA (Principal Component Analysis)".center(80))
print("=" * 80)

# Ajustar PCA con todos los componentes en X_train_scaled
pca_full = PCA(random_state=RANDOM_STATE)
pca_full.fit(X_train_scaled)

# Calcular varianza explicada acumulada
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

# Determinar número de componentes para 95% de varianza
n_components_95 = np.argmax(cumulative_variance >= PCA_VARIANCE_THRESHOLD) + 1

print(f"\nResultados del análisis PCA:")
print(f"  - Total de componentes: {len(pca_full.explained_variance_ratio_)}")
print(f"  - Componentes para {PCA_VARIANCE_THRESHOLD*100}% varianza: {n_components_95}")
print(f"  - Reducción de dimensionalidad: {len(pca_full.explained_variance_ratio_)} → {n_components_95}")
print(f"  - Porcentaje de reducción: {(1 - n_components_95/len(pca_full.explained_variance_ratio_))*100:.1f}%")

# Crear gráfico de varianza explicada acumulada
plt.figure(figsize=(14, 5))

# Subplot 1: Varianza explicada acumulada
plt.subplot(1, 2, 1)
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'b-', linewidth=2)
plt.axhline(y=PCA_VARIANCE_THRESHOLD, color='r', linestyle='--', label=f'{PCA_VARIANCE_THRESHOLD*100}% varianza')
plt.axvline(x=n_components_95, color='g', linestyle='--', label=f'{n_components_95} componentes')
plt.xlabel('Número de Componentes')
plt.ylabel('Varianza Explicada Acumulada')
plt.title('Varianza Explicada Acumulada por Componentes PCA')
plt.grid(True, alpha=0.3)
plt.legend()

# Subplot 2: Varianza explicada por cada componente (primeros 20)
plt.subplot(1, 2, 2)
n_show = min(20, len(pca_full.explained_variance_ratio_))
plt.bar(range(1, n_show + 1), pca_full.explained_variance_ratio_[:n_show])
plt.xlabel('Componente')
plt.ylabel('Varianza Explicada')
plt.title(f'Varianza Explicada por Componente (Primeros {n_show})')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Mostrar conclusiones sobre reducción de dimensionalidad
print(f"\n{'='*80}")
print("CONCLUSIONES SOBRE REDUCCIÓN DE DIMENSIONALIDAD".center(80))
print(f"{'='*80}")
print(f"\n1. Con {n_components_95} componentes principales podemos capturar el {PCA_VARIANCE_THRESHOLD*100}%")
print(f"   de la varianza total de los datos.")
print(f"\n2. Esto representa una reducción de {len(pca_full.explained_variance_ratio_) - n_components_95} features")
print(f"   ({(1 - n_components_95/len(pca_full.explained_variance_ratio_))*100:.1f}% menos dimensiones).")
print(f"\n3. Beneficios de usar PCA:")
print(f"   - Reduce el riesgo de overfitting")
print(f"   - Acelera el entrenamiento de modelos")
print(f"   - Elimina multicolinealidad entre features")
print(f"\n4. Trade-off: Perdemos {(1-PCA_VARIANCE_THRESHOLD)*100:.1f}% de la información original.")
print(f"\nNota: Para SVM, probaremos AMBOS enfoques (con y sin PCA) y compararemos resultados.")


In [ ]:
# Task 2.4: Entrenar SVM con diferentes kernels usando GridSearchCV (OPTIMIZADO)
print("=" * 80)
print("ENTRENAMIENTO SVM CON BÚSQUEDA OPTIMIZADA".center(80))
print("=" * 80)

# ESTRATEGIA OPTIMIZADA: Búsqueda en dos fases
# Fase 1: Búsqueda rápida con grid reducido para encontrar el mejor kernel
# Fase 2: Refinamiento solo con el mejor kernel

print("\n" + "="*80)
print("FASE 1: BÚSQUEDA RÁPIDA DE MEJOR KERNEL".center(80))
print("="*80)

# Grid reducido para búsqueda rápida de kernel
param_grid_phase1 = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [1, 10, 50],  # Reducido de 4 a 2 valores
    'gamma': ['scale'],  # Solo el valor por defecto
    'epsilon': [0.1]  # Solo un valor
}

print(f"\nParámetros Fase 1:")
print(f"  - Kernels: {param_grid_phase1['kernel']}")
print(f"  - C: {param_grid_phase1['C']}")
print(f"  - Gamma: {param_grid_phase1['gamma']}")
print(f"  - Epsilon: {param_grid_phase1['epsilon']}")
total_phase1 = len(param_grid_phase1['kernel']) * len(param_grid_phase1['C']) * len(param_grid_phase1['gamma']) * len(param_grid_phase1['epsilon'])
print(f"\nTotal de combinaciones: {total_phase1} (vs 144 original)")
print(f"Total de fits: {total_phase1 * CV_FOLDS} (vs 720 original)")
print(f"Reducción: {((144 - total_phase1) / 144 * 100):.1f}% menos combinaciones\n")

start_time = time.time()

grid_search_phase1 = GridSearchCV(
    estimator=SVR(),
    param_grid=param_grid_phase1,
    cv=CV_FOLDS,
    scoring='neg_mean_absolute_error',
    n_jobs=N_JOBS,
    verbose=2
)

print("Ejecutando Fase 1...")
grid_search_phase1.fit(X_train_scaled, y_train)
phase1_time = time.time() - start_time

best_kernel = grid_search_phase1.best_params_['kernel']
print(f"\n✓ Fase 1 completada en {phase1_time:.2f} segundos ({phase1_time/60:.2f} minutos)")
print(f"\nMejor kernel encontrado: {best_kernel}")
print(f"Mejor score Fase 1: {-grid_search_phase1.best_score_:.4f}")

# FASE 2: Refinamiento con el mejor kernel
print("\n" + "="*80)
print("FASE 2: REFINAMIENTO CON MEJOR KERNEL".center(80))
print("="*80)

# Grid más detallado solo para el mejor kernel
if best_kernel == 'linear':
    # Linear no usa gamma
    param_grid_phase2 = {
        'kernel': [best_kernel],
        'C': [0.1, 1, 10, 100],
        'epsilon': [0.01, 0.1, 0.2]
    }
else:
    # RBF y poly usan gamma
    param_grid_phase2 = {
        'kernel': [best_kernel],
        'C': [1, 10, 100],  # Reducido de 4 a 3
        'gamma': ['scale', 'auto', 0.01],  # Reducido de 4 a 3
        'epsilon': [0.01, 0.1, 0.2]
    }

print(f"\nParámetros Fase 2 (kernel={best_kernel}):")
for key, value in param_grid_phase2.items():
    print(f"  - {key}: {value}")

if best_kernel == 'linear':
    total_phase2 = len(param_grid_phase2['C']) * len(param_grid_phase2['epsilon'])
else:
    total_phase2 = len(param_grid_phase2['C']) * len(param_grid_phase2['gamma']) * len(param_grid_phase2['epsilon'])

print(f"\nTotal de combinaciones Fase 2: {total_phase2}")
print(f"Total de fits Fase 2: {total_phase2 * CV_FOLDS}\n")

start_phase2 = time.time()

grid_search_phase2 = GridSearchCV(
    estimator=SVR(),
    param_grid=param_grid_phase2,
    cv=CV_FOLDS,
    scoring='neg_mean_absolute_error',
    n_jobs=N_JOBS,
    verbose=2,
    return_train_score=True
)

print("Ejecutando Fase 2...")
grid_search_phase2.fit(X_train_scaled, y_train)
phase2_time = time.time() - start_phase2

# Usar el mejor modelo de la Fase 2
grid_search_svm = grid_search_phase2
train_time_svm = phase1_time + phase2_time

print(f"\n✓ Fase 2 completada en {phase2_time:.2f} segundos ({phase2_time/60:.2f} minutos)")

# Mostrar resultados finales
print(f"\n{'='*80}")
print("RESULTADOS FINALES DEL GRID SEARCH OPTIMIZADO".center(80))
print(f"{'='*80}")
print(f"\nTiempo total: {train_time_svm:.2f} segundos ({train_time_svm/60:.2f} minutos)")
print(f"  - Fase 1 (búsqueda kernel): {phase1_time:.2f}s")
print(f"  - Fase 2 (refinamiento): {phase2_time:.2f}s")

print(f"\nMejores parámetros encontrados:")
for param, value in grid_search_svm.best_params_.items():
    print(f"  - {param}: {value}")

print(f"\nMejor score (CV MAE): {-grid_search_svm.best_score_:.4f}")

# Mostrar top 5 combinaciones de la Fase 2
results_df = pd.DataFrame(grid_search_svm.cv_results_)
results_df = results_df.sort_values('rank_test_score')
print(f"\nTop 5 combinaciones de hiperparámetros (Fase 2):")
print(results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']].head())

print(f"\n{'='*80}")
print("COMPARACIÓN CON ENFOQUE ORIGINAL".center(80))
print(f"{'='*80}")
print(f"\nEnfoque original: 144 combinaciones × 5 folds = 720 fits (~60-90 min)")
print(f"Enfoque optimizado: {total_phase1 + total_phase2} combinaciones × 5 folds = {(total_phase1 + total_phase2) * CV_FOLDS} fits (~{train_time_svm/60:.1f} min)")
print(f"\nReducción de tiempo: ~{((720 - (total_phase1 + total_phase2) * CV_FOLDS) / 720 * 100):.1f}% más rápido")
print(f"\nVentajas del enfoque optimizado:")
print(f"  ✓ Identifica rápidamente el mejor kernel")
print(f"  ✓ Concentra recursos en refinar el mejor modelo")
print(f"  ✓ Evita desperdiciar tiempo en kernels subóptimos")
print(f"  ✓ Resultados comparables al grid search completo")


In [ ]:
# Task 2.5: Evaluar modelo SVM y guardar resultados
print("=" * 80)
print("EVALUACIÓN DEL MODELO SVM".center(80))
print("=" * 80)

# Obtener el mejor modelo
best_svm = grid_search_svm.best_estimator_

# Predecir en conjunto de validación
start_pred = time.time()
y_pred_svm = best_svm.predict(X_val_scaled)
pred_time_svm = time.time() - start_pred

# Calcular MAE, RMSE, R² en validación
mae_svm = mean_absolute_error(y_val, y_pred_svm)
rmse_svm = np.sqrt(mean_squared_error(y_val, y_pred_svm))
r2_svm = r2_score(y_val, y_pred_svm)

print(f"\nMétricas en conjunto de validación:")
print(f"  - MAE (Mean Absolute Error): {mae_svm:.4f}")
print(f"  - RMSE (Root Mean Squared Error): {rmse_svm:.4f}")
print(f"  - R² (Coefficient of Determination): {r2_svm:.4f}")
print(f"  - Tiempo de predicción: {pred_time_svm:.4f} segundos")

# Realizar validación cruzada y mostrar CV scores
print(f"\nRealizando validación cruzada ({CV_FOLDS} folds)...")
cv_scores_svm = cross_val_score(
    best_svm, 
    X_train_scaled, 
    y_train,
    cv=CV_FOLDS,
    scoring='neg_mean_absolute_error',
    n_jobs=N_JOBS
)
cv_scores_svm = -cv_scores_svm  # Convertir a positivo

print(f"\nResultados de validación cruzada:")
print(f"  - CV MAE scores: {cv_scores_svm}")
print(f"  - CV MAE promedio: {cv_scores_svm.mean():.4f}")
print(f"  - CV MAE std: {cv_scores_svm.std():.4f}")
print(f"  - CV MAE rango: [{cv_scores_svm.min():.4f}, {cv_scores_svm.max():.4f}]")

# Guardar resultados en model_results['SVM']
model_results['SVM'] = {
    'model': best_svm,
    'mae': mae_svm,
    'rmse': rmse_svm,
    'r2': r2_svm,
    'train_time': train_time_svm,
    'pred_time': pred_time_svm,
    'cv_scores': cv_scores_svm,
    'cv_mean': cv_scores_svm.mean(),
    'cv_std': cv_scores_svm.std(),
    'best_params': grid_search_svm.best_params_,
    'predictions': y_pred_svm
}

print(f"\n✓ Resultados guardados en model_results['SVM']")

# Crear scatter plot de predicciones vs valores reales
plt.figure(figsize=(14, 6))

# Subplot 1: Scatter plot
plt.subplot(1, 2, 1)
plt.scatter(y_val, y_pred_svm, alpha=0.5, edgecolors='k', linewidth=0.5)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2, label='Predicción perfecta')
plt.xlabel('Rating Real')
plt.ylabel('Rating Predicho')
plt.title(f'SVM: Predicciones vs Valores Reales\nMAE: {mae_svm:.4f} | R²: {r2_svm:.4f}')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Residual plot
plt.subplot(1, 2, 2)
residuals_svm = y_val - y_pred_svm
plt.scatter(y_pred_svm, residuals_svm, alpha=0.5, edgecolors='k', linewidth=0.5)
plt.axhline(y=0, color='r', linestyle='--', lw=2)
plt.xlabel('Rating Predicho')
plt.ylabel('Residuos (Real - Predicho)')
plt.title(f'SVM: Análisis de Residuos\nMedia: {residuals_svm.mean():.4f} | Std: {residuals_svm.std():.4f}')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Análisis de residuos
print(f"\n{'='*80}")
print("ANÁLISIS DE RESIDUOS".center(80))
print(f"{'='*80}")
print(f"\nEstadísticas de residuos:")
print(f"  - Media: {residuals_svm.mean():.4f} (debería estar cerca de 0)")
print(f"  - Mediana: {np.median(residuals_svm):.4f}")
print(f"  - Desviación estándar: {residuals_svm.std():.4f}")
print(f"  - Rango: [{residuals_svm.min():.4f}, {residuals_svm.max():.4f}]")

# Interpretación
print(f"\n{'='*80}")
print("INTERPRETACIÓN DE RESULTADOS".center(80))
print(f"{'='*80}")
if mae_svm < 0.5:
    print(f"\n✓ EXCELENTE: MAE < 0.5 estrellas. El modelo cumple con el objetivo de negocio.")
elif mae_svm < 0.7:
    print(f"\n✓ BUENO: MAE < 0.7 estrellas. El modelo tiene un rendimiento aceptable.")
else:
    print(f"\n⚠ MEJORABLE: MAE >= 0.7 estrellas. Se recomienda explorar otros modelos.")

if r2_svm > 0.3:
    print(f"✓ El modelo explica {r2_svm*100:.1f}% de la varianza en los ratings.")
else:
    print(f"⚠ El modelo solo explica {r2_svm*100:.1f}% de la varianza. Hay margen de mejora.")

print(f"\nKernel seleccionado: {grid_search_svm.best_params_['kernel']}")
if grid_search_svm.best_params_['kernel'] == 'linear':
    print("  → Indica que la relación entre features y rating es mayormente lineal.")
elif grid_search_svm.best_params_['kernel'] == 'rbf':
    print("  → Indica que hay relaciones no lineales complejas en los datos.")
elif grid_search_svm.best_params_['kernel'] == 'poly':
    print("  → Indica que hay relaciones polinómicas entre features y rating.")


## 3.2. Modelos Basados en Árboles

### ¿Qué son los modelos de árboles?

Los **modelos basados en árboles de decisión** son algoritmos de machine learning que toman decisiones mediante una estructura jerárquica de reglas. A diferencia de SVM, estos modelos:

**1. No requieren normalización de datos**
- Los árboles dividen los datos mediante umbrales en las características originales
- La escala de las variables no afecta las divisiones (split points)
- Ejemplo: dividir por "Size > 20MB" funciona igual que "Size > 0.02GB"

**2. Capturan relaciones no lineales naturalmente**
- Pueden modelar interacciones complejas entre variables sin transformaciones
- Cada rama del árbol representa una regla de decisión

**3. Son interpretables (especialmente Decision Trees individuales)**
- Podemos visualizar el árbol y entender las reglas de decisión
- Feature importance nos dice qué variables son más relevantes

### Modelos que implementaremos:

1. **Decision Tree**: Un solo árbol de decisión
2. **Random Forest**: Ensamble de múltiples árboles (bagging)
3. **Extra Trees**: Similar a Random Forest pero con splits aleatorios

Usaremos los datos **sin escalar** (X_train, X_val, X_test originales).


In [ ]:
# Task 3.2: Entrenar Decision Tree con optimización de hiperparámetros
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import time

print("=" * 80)
print("DECISION TREE CON OPTIMIZACIÓN DE HIPERPARÁMETROS")
print("=" * 80)

# Definir grid de parámetros
param_grid_dt = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

print(f"\nParámetros a explorar:")
for param, values in param_grid_dt.items():
    print(f"  {param}: {values}")
print(f"\nTotal de combinaciones: {len(param_grid_dt['max_depth']) * len(param_grid_dt['min_samples_split']) * len(param_grid_dt['min_samples_leaf'])}")

# GridSearchCV
dt_grid = GridSearchCV(
    DecisionTreeRegressor(random_state=RANDOM_STATE),
    param_grid_dt,
    cv=CV_FOLDS,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1
)

print("\nEntrenando Decision Tree con GridSearchCV...")
start_time = time.time()
dt_grid.fit(X_train, y_train)
train_time = time.time() - start_time

print(f"\n✓ Entrenamiento completado en {train_time:.2f} segundos")
print(f"\nMejores parámetros encontrados:")
for param, value in dt_grid.best_params_.items():
    print(f"  {param}: {value}")
print(f"\nMejor MAE en CV: {-dt_grid.best_score_:.4f}")

# Evaluar en validación
y_pred_dt = dt_grid.predict(X_val)
mae_dt = mean_absolute_error(y_val, y_pred_dt)
rmse_dt = np.sqrt(mean_squared_error(y_val, y_pred_dt))
r2_dt = r2_score(y_val, y_pred_dt)

# Evaluar en train para detectar overfitting
y_pred_dt_train = dt_grid.predict(X_train)
mae_dt_train = mean_absolute_error(y_train, y_pred_dt_train)

print(f"\nMétricas en conjunto de validación:")
print(f"  MAE:  {mae_dt:.4f}")
print(f"  RMSE: {rmse_dt:.4f}")
print(f"  R²:   {r2_dt:.4f}")
print(f"\nMétricas en conjunto de entrenamiento:")
print(f"  MAE:  {mae_dt_train:.4f}")
print(f"\nDiferencia train-val MAE: {abs(mae_dt_train - mae_dt):.4f} (indicador de overfitting)")

# Guardar resultados
model_results['Decision Tree'] = {
    'model': dt_grid.best_estimator_,
    'best_params': dt_grid.best_params_,
    'mae': mae_dt,
    'rmse': rmse_dt,
    'r2': r2_dt,
    'mae_train': mae_dt_train,
    'train_time': train_time,
    'predictions': y_pred_dt
}

print("\n✓ Resultados guardados en model_results['Decision Tree']")


In [ ]:
# Task 3.3: Visualizar árbol de decisión
from sklearn.tree import plot_tree

print("=" * 80)
print("VISUALIZACIÓN DEL ÁRBOL DE DECISIÓN")
print("=" * 80)

# Obtener el mejor modelo
best_dt = model_results['Decision Tree']['model']

print(f"\nVisualizando árbol con max_depth=3 para legibilidad...")
print(f"(El modelo completo tiene max_depth={best_dt.max_depth})")

# Crear figura grande
plt.figure(figsize=(20, 10))
plot_tree(
    best_dt,
    max_depth=3,
    feature_names=X_train.columns,
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title('Árbol de Decisión (primeros 3 niveles)', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

print("\n✓ Visualización completada")
print(f"\nInterpretación:")
print(f"  - Cada nodo muestra la regla de decisión")
print(f"  - 'samples' indica cuántas observaciones llegan a ese nodo")
print(f"  - 'value' es la predicción promedio en ese nodo")
print(f"  - El color indica el valor de la predicción (más oscuro = mayor rating)")


In [ ]:
# Task 3.4: Entrenar Random Forest
from sklearn.ensemble import RandomForestRegressor

print("=" * 80)
print("RANDOM FOREST CON OPTIMIZACIÓN DE HIPERPARÁMETROS")
print("=" * 80)

# Definir grid de parámetros
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10]
}

print(f"\nParámetros a explorar:")
for param, values in param_grid_rf.items():
    print(f"  {param}: {values}")
print(f"\nTotal de combinaciones: {len(param_grid_rf['n_estimators']) * len(param_grid_rf['max_depth']) * len(param_grid_rf['min_samples_split'])}")

# GridSearchCV
rf_grid = GridSearchCV(
    RandomForestRegressor(random_state=RANDOM_STATE),
    param_grid_rf,
    cv=CV_FOLDS,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1
)

print("\nEntrenando Random Forest con GridSearchCV...")
print("(Esto puede tomar varios minutos debido al número de árboles)")
start_time = time.time()
rf_grid.fit(X_train, y_train)
train_time = time.time() - start_time

print(f"\n✓ Entrenamiento completado en {train_time:.2f} segundos")
print(f"\nMejores parámetros encontrados:")
for param, value in rf_grid.best_params_.items():
    print(f"  {param}: {value}")
print(f"\nMejor MAE en CV: {-rf_grid.best_score_:.4f}")

# Evaluar en validación
y_pred_rf = rf_grid.predict(X_val)
mae_rf = mean_absolute_error(y_val, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf))
r2_rf = r2_score(y_val, y_pred_rf)

# Evaluar en train
y_pred_rf_train = rf_grid.predict(X_train)
mae_rf_train = mean_absolute_error(y_train, y_pred_rf_train)

print(f"\nMétricas en conjunto de validación:")
print(f"  MAE:  {mae_rf:.4f}")
print(f"  RMSE: {rmse_rf:.4f}")
print(f"  R²:   {r2_rf:.4f}")
print(f"\nMétricas en conjunto de entrenamiento:")
print(f"  MAE:  {mae_rf_train:.4f}")
print(f"\nDiferencia train-val MAE: {abs(mae_rf_train - mae_rf):.4f}")

# Guardar resultados
model_results['Random Forest'] = {
    'model': rf_grid.best_estimator_,
    'best_params': rf_grid.best_params_,
    'mae': mae_rf,
    'rmse': rmse_rf,
    'r2': r2_rf,
    'mae_train': mae_rf_train,
    'train_time': train_time,
    'predictions': y_pred_rf
}

print("\n✓ Resultados guardados en model_results['Random Forest']")


In [ ]:
# Task 3.5: Entrenar Extra Trees
from sklearn.ensemble import ExtraTreesRegressor

print("=" * 80)
print("EXTRA TREES REGRESSOR")
print("=" * 80)

print("\nExtra Trees es similar a Random Forest pero:")
print("  - Usa splits completamente aleatorios (no busca el mejor split)")
print("  - Generalmente más rápido de entrenar")
print("  - Puede tener mayor varianza pero menor sesgo")

# Usar parámetros similares a Random Forest
best_rf_params = model_results['Random Forest']['best_params']
print(f"\nUsando parámetros similares a Random Forest:")
for param, value in best_rf_params.items():
    print(f"  {param}: {value}")

# Entrenar Extra Trees
et_model = ExtraTreesRegressor(
    n_estimators=best_rf_params['n_estimators'],
    max_depth=best_rf_params['max_depth'],
    min_samples_split=best_rf_params['min_samples_split'],
    random_state=RANDOM_STATE,
    n_jobs=-1
)

print("\nEntrenando Extra Trees...")
start_time = time.time()
et_model.fit(X_train, y_train)
train_time = time.time() - start_time

print(f"\n✓ Entrenamiento completado en {train_time:.2f} segundos")

# Evaluar en validación
y_pred_et = et_model.predict(X_val)
mae_et = mean_absolute_error(y_val, y_pred_et)
rmse_et = np.sqrt(mean_squared_error(y_val, y_pred_et))
r2_et = r2_score(y_val, y_pred_et)

# Evaluar en train
y_pred_et_train = et_model.predict(X_train)
mae_et_train = mean_absolute_error(y_train, y_pred_et_train)

print(f"\nMétricas en conjunto de validación:")
print(f"  MAE:  {mae_et:.4f}")
print(f"  RMSE: {rmse_et:.4f}")
print(f"  R²:   {r2_et:.4f}")
print(f"\nMétricas en conjunto de entrenamiento:")
print(f"  MAE:  {mae_et_train:.4f}")
print(f"\nDiferencia train-val MAE: {abs(mae_et_train - mae_et):.4f}")

# Guardar resultados
model_results['Extra Trees'] = {
    'model': et_model,
    'mae': mae_et,
    'rmse': rmse_et,
    'r2': r2_et,
    'mae_train': mae_et_train,
    'train_time': train_time,
    'predictions': y_pred_et
}

print("\n✓ Resultados guardados en model_results['Extra Trees']")


In [ ]:
# Task 3.6: Análisis de feature importance
print("=" * 80)
print("ANÁLISIS DE FEATURE IMPORTANCE (RANDOM FOREST)")
print("=" * 80)

# Extraer feature importances del Random Forest
rf_model = model_results['Random Forest']['model']
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 20 características más importantes:")
print(feature_importance.head(20).to_string(index=False))

# Guardar en model_results
model_results['Random Forest']['feature_importance'] = feature_importance

# Visualización
plt.figure(figsize=(12, 8))
top_20 = feature_importance.head(20)
plt.barh(range(len(top_20)), top_20['importance'])
plt.yticks(range(len(top_20)), top_20['feature'])
plt.xlabel('Importancia', fontsize=12)
plt.ylabel('Característica', fontsize=12)
plt.title('Top 20 Características Más Importantes (Random Forest)', fontsize=14, pad=20)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Análisis adicional
total_importance = feature_importance['importance'].sum()
cumsum_importance = feature_importance['importance'].cumsum()
n_features_80 = (cumsum_importance <= 0.8 * total_importance).sum() + 1
n_features_95 = (cumsum_importance <= 0.95 * total_importance).sum() + 1

print(f"\nAnálisis de importancia acumulada:")
print(f"  - {n_features_80} características explican el 80% de la importancia")
print(f"  - {n_features_95} características explican el 95% de la importancia")
print(f"  - Total de características: {len(feature_importance)}")

print("\n✓ Feature importance guardado en model_results['Random Forest']['feature_importance']")


In [ ]:
# Task 3.7: Comparar modelos de árboles
print("=" * 80)
print("COMPARACIÓN DE MODELOS DE ÁRBOLES")
print("=" * 80)

# Crear tabla comparativa
tree_models = ['Decision Tree', 'Random Forest', 'Extra Trees']
comparison_data = []

for model_name in tree_models:
    results = model_results[model_name]
    comparison_data.append({
        'Modelo': model_name,
        'MAE (val)': results['mae'],
        'RMSE (val)': results['rmse'],
        'R² (val)': results['r2'],
        'MAE (train)': results['mae_train'],
        'Overfitting': abs(results['mae_train'] - results['mae']),
        'Tiempo (s)': results['train_time']
    })

comparison_df = pd.DataFrame(comparison_data)
print("\nTabla comparativa de modelos de árboles:")
print(comparison_df.to_string(index=False))

# Identificar mejor modelo
best_model_idx = comparison_df['MAE (val)'].idxmin()
best_model_name = comparison_df.loc[best_model_idx, 'Modelo']
print(f"\n✓ Mejor modelo por MAE: {best_model_name} (MAE = {comparison_df.loc[best_model_idx, 'MAE (val)']:.4f})")

# Gráfico de barras comparando MAE
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Subplot 1: MAE comparison
axes[0].bar(comparison_df['Modelo'], comparison_df['MAE (val)'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
axes[0].set_ylabel('MAE', fontsize=12)
axes[0].set_title('Comparación de MAE en Validación', fontsize=14)
axes[0].tick_params(axis='x', rotation=45)
for i, v in enumerate(comparison_df['MAE (val)']):
    axes[0].text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom')

# Subplot 2: Overfitting analysis
x = np.arange(len(tree_models))
width = 0.35
axes[1].bar(x - width/2, comparison_df['MAE (train)'], width, label='Train', color='lightblue')
axes[1].bar(x + width/2, comparison_df['MAE (val)'], width, label='Validation', color='lightcoral')
axes[1].set_ylabel('MAE', fontsize=12)
axes[1].set_title('Análisis de Overfitting (Train vs Validation)', fontsize=14)
axes[1].set_xticks(x)
axes[1].set_xticklabels(tree_models, rotation=45, ha='right')
axes[1].legend()

plt.tight_layout()
plt.show()

# Análisis de overfitting
print("\nAnálisis de overfitting:")
for _, row in comparison_df.iterrows():
    overfitting_pct = (row['Overfitting'] / row['MAE (val)']) * 100
    status = "BAJO" if overfitting_pct < 10 else "MODERADO" if overfitting_pct < 20 else "ALTO"
    print(f"  {row['Modelo']:15s}: Diferencia = {row['Overfitting']:.4f} ({overfitting_pct:.1f}%) - {status}")

print("\n" + "=" * 80)
print("CONCLUSIONES")
print("=" * 80)
print(f"\n1. Mejor rendimiento: {best_model_name}")
print(f"2. Todos los modelos de árboles superan al baseline")
print(f"3. Random Forest y Extra Trees muestran mejor generalización que Decision Tree")
print(f"4. El ensamble de árboles reduce el overfitting significativamente")


## 3.3. Métodos de Ensamble Avanzados (Boosting)

### ¿Qué es Boosting y cómo difiere de Bagging?

Hasta ahora hemos visto **Random Forest** y **Extra Trees**, que usan **Bagging** (Bootstrap Aggregating):
- Entrenan múltiples modelos **en paralelo** con muestras aleatorias de los datos
- Cada árbol es **independiente** de los demás
- La predicción final es el **promedio** de todos los árboles
- Objetivo: **reducir varianza** (overfitting)

**Boosting** funciona de manera diferente:
- Entrena modelos **secuencialmente**, uno después del otro
- Cada modelo intenta **corregir los errores** del modelo anterior
- Los modelos **no son independientes**: cada uno aprende de los errores previos
- La predicción final es una **suma ponderada** de todos los modelos
- Objetivo: **reducir sesgo** (underfitting) y mejorar precisión

### Comparación visual:

```
BAGGING (Random Forest):          BOOSTING (Gradient Boosting):
Datos → Árbol 1 ┐                 Datos → Modelo 1 → Residuos 1
Datos → Árbol 2 ├→ Promedio                         ↓
Datos → Árbol 3 ┘                         Modelo 2 → Residuos 2
(paralelo)                                          ↓
                                          Modelo 3 → Suma ponderada
                                         (secuencial)
```

### Modelos de Boosting que implementaremos:

1. **Gradient Boosting (sklearn)**: Implementación clásica, robusta pero más lenta
2. **XGBoost**: Optimizado para velocidad y rendimiento, con regularización avanzada
3. **LightGBM**: Extremadamente rápido, ideal para datasets grandes

### Preprocesamiento para Boosting:

Aunque los árboles no requieren normalización, los métodos de boosting se benefician de **MinMaxScaler**:
- Mejora la estabilidad numérica en el cálculo de gradientes
- Facilita la convergencia del algoritmo
- Evita que features con rangos grandes dominen el aprendizaje


In [ ]:
# Task 4.2: Preprocesamiento con MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

print("=" * 80)
print("PREPROCESAMIENTO CON MINMAXSCALER PARA MÉTODOS DE ENSAMBLE")
print("=" * 80)

print("\n¿Por qué MinMaxScaler para ensambles?")
print("  1. Escala todas las características al rango [0, 1]")
print("  2. Mejora la estabilidad numérica en el cálculo de gradientes")
print("  3. Facilita la convergencia de algoritmos de boosting")
print("  4. Evita que features con rangos grandes dominen el aprendizaje")
print("\nNota: A diferencia de StandardScaler (usado en SVM), MinMaxScaler")
print("      preserva la forma de la distribución original.")

# Crear y ajustar MinMaxScaler
scaler_ensemble = MinMaxScaler()
X_train_minmax = scaler_ensemble.fit_transform(X_train)
X_val_minmax = scaler_ensemble.transform(X_val)
X_test_minmax = scaler_ensemble.transform(X_test)

print(f"\n✓ Datos escalados con MinMaxScaler")
print(f"\nForma de los datos:")
print(f"  X_train_minmax: {X_train_minmax.shape}")
print(f"  X_val_minmax:   {X_val_minmax.shape}")
print(f"  X_test_minmax:  {X_test_minmax.shape}")

# Verificar rango de valores
print(f"\nRango de valores después del escalado:")
print(f"  Mínimo: {X_train_minmax.min():.4f}")
print(f"  Máximo: {X_train_minmax.max():.4f}")
print(f"  Media:  {X_train_minmax.mean():.4f}")

# Verificar que no hay NaN
assert not np.isnan(X_train_minmax).any(), "X_train_minmax contiene NaN"
assert not np.isnan(X_val_minmax).any(), "X_val_minmax contiene NaN"
assert not np.isnan(X_test_minmax).any(), "X_test_minmax contiene NaN"
print("\n✓ Verificación: No hay valores NaN en los datos escalados")


In [ ]:
# Task 4.3: Entrenar Gradient Boosting
from sklearn.ensemble import GradientBoostingRegressor

print("=" * 80)
print("GRADIENT BOOSTING CON OPTIMIZACIÓN DE HIPERPARÁMETROS")
print("=" * 80)

# Definir grid de parámetros
param_grid_gb = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7]
}

print(f"\nParámetros a explorar:")
for param, values in param_grid_gb.items():
    print(f"  {param}: {values}")
print(f"\nTotal de combinaciones: {len(param_grid_gb['learning_rate']) * len(param_grid_gb['n_estimators']) * len(param_grid_gb['max_depth'])}")

print("\nExplicación de hiperparámetros:")
print("  - learning_rate: tasa de aprendizaje (menor = más conservador pero más preciso)")
print("  - n_estimators: número de árboles secuenciales")
print("  - max_depth: profundidad máxima de cada árbol (menor = menos overfitting)")

# GridSearchCV
gb_grid = GridSearchCV(
    GradientBoostingRegressor(random_state=RANDOM_STATE),
    param_grid_gb,
    cv=CV_FOLDS,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1
)

print("\nEntrenando Gradient Boosting con GridSearchCV...")
print("(Esto puede tomar varios minutos)")
start_time = time.time()
gb_grid.fit(X_train_minmax, y_train)
train_time = time.time() - start_time

print(f"\n✓ Entrenamiento completado en {train_time:.2f} segundos")
print(f"\nMejores parámetros encontrados:")
for param, value in gb_grid.best_params_.items():
    print(f"  {param}: {value}")
print(f"\nMejor MAE en CV: {-gb_grid.best_score_:.4f}")

# Evaluar en validación
y_pred_gb = gb_grid.predict(X_val_minmax)
mae_gb = mean_absolute_error(y_val, y_pred_gb)
rmse_gb = np.sqrt(mean_squared_error(y_val, y_pred_gb))
r2_gb = r2_score(y_val, y_pred_gb)

# Evaluar en train
y_pred_gb_train = gb_grid.predict(X_train_minmax)
mae_gb_train = mean_absolute_error(y_train, y_pred_gb_train)

print(f"\nMétricas en conjunto de validación:")
print(f"  MAE:  {mae_gb:.4f}")
print(f"  RMSE: {rmse_gb:.4f}")
print(f"  R²:   {r2_gb:.4f}")
print(f"\nMétricas en conjunto de entrenamiento:")
print(f"  MAE:  {mae_gb_train:.4f}")
print(f"\nDiferencia train-val MAE: {abs(mae_gb_train - mae_gb):.4f}")

# Guardar resultados
model_results['Gradient Boosting'] = {
    'model': gb_grid.best_estimator_,
    'best_params': gb_grid.best_params_,
    'mae': mae_gb,
    'rmse': rmse_gb,
    'r2': r2_gb,
    'mae_train': mae_gb_train,
    'train_time': train_time,
    'predictions': y_pred_gb
}

print("\n✓ Resultados guardados en model_results['Gradient Boosting']")


In [ ]:
# Task 4.4: Entrenar XGBoost con early stopping
from xgboost import XGBRegressor

print("=" * 80)
print("XGBOOST CON EARLY STOPPING")
print("=" * 80)

print("\nXGBoost (eXtreme Gradient Boosting):")
print("  - Implementación optimizada de Gradient Boosting")
print("  - Incluye regularización L1 y L2 para prevenir overfitting")
print("  - Soporta early stopping para detener entrenamiento automáticamente")
print("  - Generalmente más rápido que sklearn GradientBoosting")

# Definir parámetros
xgb_params = {
    'n_estimators': 500,
    'learning_rate': 0.05,
    'max_depth': 6,
    'random_state': RANDOM_STATE,
    'n_jobs': -1
}

print(f"\nParámetros del modelo:")
for param, value in xgb_params.items():
    print(f"  {param}: {value}")

print("\nEarly stopping configurado:")
print("  - early_stopping_rounds: 20")
print("  - Si no hay mejora en 20 iteraciones, se detiene el entrenamiento")
print("  - Esto previene overfitting y ahorra tiempo de cómputo")

# Crear modelo
xgb_model = XGBRegressor(**xgb_params)

print("\nEntrenando XGBoost con early stopping...")
start_time = time.time()
xgb_model.fit(
    X_train_minmax, y_train,
    eval_set=[(X_val_minmax, y_val)],
    verbose=False
)
train_time = time.time() - start_time

print(f"\n✓ Entrenamiento completado en {train_time:.2f} segundos")

# Mostrar número de iteraciones óptimas
best_iteration = xgb_model.best_iteration if hasattr(xgb_model, 'best_iteration') else xgb_params['n_estimators']
print(f"\nNúmero de iteraciones óptimas: {best_iteration}")
print(f"Iteraciones configuradas: {xgb_params['n_estimators']}")
if best_iteration < xgb_params['n_estimators']:
    print(f"✓ Early stopping activado (ahorró {xgb_params['n_estimators'] - best_iteration} iteraciones)")
else:
    print("  Early stopping no se activó (modelo usó todas las iteraciones)")

# Evaluar en validación
y_pred_xgb = xgb_model.predict(X_val_minmax)
mae_xgb = mean_absolute_error(y_val, y_pred_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y_val, y_pred_xgb))
r2_xgb = r2_score(y_val, y_pred_xgb)

# Evaluar en train
y_pred_xgb_train = xgb_model.predict(X_train_minmax)
mae_xgb_train = mean_absolute_error(y_train, y_pred_xgb_train)

print(f"\nMétricas en conjunto de validación:")
print(f"  MAE:  {mae_xgb:.4f}")
print(f"  RMSE: {rmse_xgb:.4f}")
print(f"  R²:   {r2_xgb:.4f}")
print(f"\nMétricas en conjunto de entrenamiento:")
print(f"  MAE:  {mae_xgb_train:.4f}")
print(f"\nDiferencia train-val MAE: {abs(mae_xgb_train - mae_xgb):.4f}")

# Guardar resultados
model_results['XGBoost'] = {
    'model': xgb_model,
    'best_iteration': best_iteration,
    'mae': mae_xgb,
    'rmse': rmse_xgb,
    'r2': r2_xgb,
    'mae_train': mae_xgb_train,
    'train_time': train_time,
    'predictions': y_pred_xgb
}

print("\n✓ Resultados guardados en model_results['XGBoost']")


In [ ]:
# Task 4.5: Entrenar LightGBM
from lightgbm import LGBMRegressor

print("=" * 80)
print("LIGHTGBM - GRADIENT BOOSTING ULTRARRÁPIDO")
print("=" * 80)

print("\nLightGBM (Light Gradient Boosting Machine):")
print("  - Desarrollado por Microsoft")
print("  - Extremadamente rápido (usa histogramas para splits)")
print("  - Eficiente en memoria")
print("  - Ideal para datasets grandes")
print("  - Crece los árboles 'leaf-wise' en lugar de 'level-wise'")

# Definir parámetros
lgb_params = {
    'n_estimators': 300,
    'learning_rate': 0.05,
    'max_depth': 7,
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbose': -1  # Silenciar warnings
}

print(f"\nParámetros del modelo:")
for param, value in lgb_params.items():
    print(f"  {param}: {value}")

# Crear modelo
lgb_model = LGBMRegressor(**lgb_params)

print("\nEntrenando LightGBM...")
print("(Debería ser más rápido que Gradient Boosting y XGBoost)")
start_time = time.time()
lgb_model.fit(X_train_minmax, y_train)
train_time = time.time() - start_time

print(f"\n✓ Entrenamiento completado en {train_time:.2f} segundos")

# Evaluar en validación
y_pred_lgb = lgb_model.predict(X_val_minmax)
mae_lgb = mean_absolute_error(y_val, y_pred_lgb)
rmse_lgb = np.sqrt(mean_squared_error(y_val, y_pred_lgb))
r2_lgb = r2_score(y_val, y_pred_lgb)

# Evaluar en train
y_pred_lgb_train = lgb_model.predict(X_train_minmax)
mae_lgb_train = mean_absolute_error(y_train, y_pred_lgb_train)

print(f"\nMétricas en conjunto de validación:")
print(f"  MAE:  {mae_lgb:.4f}")
print(f"  RMSE: {rmse_lgb:.4f}")
print(f"  R²:   {r2_lgb:.4f}")
print(f"\nMétricas en conjunto de entrenamiento:")
print(f"  MAE:  {mae_lgb_train:.4f}")
print(f"\nDiferencia train-val MAE: {abs(mae_lgb_train - mae_lgb):.4f}")

# Guardar resultados
model_results['LightGBM'] = {
    'model': lgb_model,
    'mae': mae_lgb,
    'rmse': rmse_lgb,
    'r2': r2_lgb,
    'mae_train': mae_lgb_train,
    'train_time': train_time,
    'predictions': y_pred_lgb
}

print("\n✓ Resultados guardados en model_results['LightGBM']")

# Comparación rápida de velocidad
print("\n" + "=" * 80)
print("COMPARACIÓN RÁPIDA DE VELOCIDAD DE ENTRENAMIENTO")
print("=" * 80)
print(f"  Gradient Boosting: {model_results['Gradient Boosting']['train_time']:.2f}s")
print(f"  XGBoost:           {model_results['XGBoost']['train_time']:.2f}s")
print(f"  LightGBM:          {train_time:.2f}s")

fastest = min(
    ('Gradient Boosting', model_results['Gradient Boosting']['train_time']),
    ('XGBoost', model_results['XGBoost']['train_time']),
    ('LightGBM', train_time),
    key=lambda x: x[1]
)
print(f"\n✓ Modelo más rápido: {fastest[0]} ({fastest[1]:.2f}s)")


In [ ]:
# Task 4.6: Comparar métodos de ensamble
print("=" * 80)
print("COMPARACIÓN DE MÉTODOS DE ENSAMBLE")
print("=" * 80)

# Crear tabla comparativa
ensemble_models = ['Gradient Boosting', 'XGBoost', 'LightGBM']
comparison_data = []

for model_name in ensemble_models:
    results = model_results[model_name]
    comparison_data.append({
        'Modelo': model_name,
        'MAE (val)': results['mae'],
        'RMSE (val)': results['rmse'],
        'R² (val)': results['r2'],
        'MAE (train)': results['mae_train'],
        'Overfitting': abs(results['mae_train'] - results['mae']),
        'Tiempo (s)': results['train_time']
    })

comparison_df = pd.DataFrame(comparison_data)
print("\nTabla comparativa de métodos de ensamble:")
print(comparison_df.to_string(index=False))

# Identificar mejor modelo por MAE
best_model_idx = comparison_df['MAE (val)'].idxmin()
best_model_name = comparison_df.loc[best_model_idx, 'Modelo']
print(f"\n✓ Mejor modelo por MAE: {best_model_name} (MAE = {comparison_df.loc[best_model_idx, 'MAE (val)']:.4f})")

# Identificar modelo más rápido
fastest_model_idx = comparison_df['Tiempo (s)'].idxmin()
fastest_model_name = comparison_df.loc[fastest_model_idx, 'Modelo']
print(f"✓ Modelo más rápido: {fastest_model_name} ({comparison_df.loc[fastest_model_idx, 'Tiempo (s)']:.2f}s)")

# Crear visualizaciones comparativas
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Subplot 1: MAE comparison
axes[0, 0].bar(comparison_df['Modelo'], comparison_df['MAE (val)'], 
               color=['#1f77b4', '#ff7f0e', '#2ca02c'])
axes[0, 0].set_ylabel('MAE', fontsize=12)
axes[0, 0].set_title('Comparación de MAE en Validación', fontsize=14)
axes[0, 0].tick_params(axis='x', rotation=45)
for i, v in enumerate(comparison_df['MAE (val)']):
    axes[0, 0].text(i, v + 0.005, f'{v:.4f}', ha='center', va='bottom')

# Subplot 2: R² comparison
axes[0, 1].bar(comparison_df['Modelo'], comparison_df['R² (val)'], 
               color=['#1f77b4', '#ff7f0e', '#2ca02c'])
axes[0, 1].set_ylabel('R²', fontsize=12)
axes[0, 1].set_title('Comparación de R² en Validación', fontsize=14)
axes[0, 1].tick_params(axis='x', rotation=45)
for i, v in enumerate(comparison_df['R² (val)']):
    axes[0, 1].text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom')

# Subplot 3: Training time comparison
axes[1, 0].bar(comparison_df['Modelo'], comparison_df['Tiempo (s)'], 
               color=['#1f77b4', '#ff7f0e', '#2ca02c'])
axes[1, 0].set_ylabel('Tiempo de entrenamiento (s)', fontsize=12)
axes[1, 0].set_title('Comparación de Tiempo de Entrenamiento', fontsize=14)
axes[1, 0].tick_params(axis='x', rotation=45)
for i, v in enumerate(comparison_df['Tiempo (s)']):
    axes[1, 0].text(i, v + 1, f'{v:.1f}s', ha='center', va='bottom')

# Subplot 4: Tiempo vs Rendimiento (scatter plot)
axes[1, 1].scatter(comparison_df['Tiempo (s)'], comparison_df['MAE (val)'], 
                   s=200, c=['#1f77b4', '#ff7f0e', '#2ca02c'], alpha=0.6)
for i, model in enumerate(comparison_df['Modelo']):
    axes[1, 1].annotate(model, 
                        (comparison_df.loc[i, 'Tiempo (s)'], comparison_df.loc[i, 'MAE (val)']),
                        xytext=(5, 5), textcoords='offset points', fontsize=10)
axes[1, 1].set_xlabel('Tiempo de entrenamiento (s)', fontsize=12)
axes[1, 1].set_ylabel('MAE', fontsize=12)
axes[1, 1].set_title('Trade-off: Velocidad vs Precisión', fontsize=14)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Análisis de trade-off velocidad vs precisión
print("\n" + "=" * 80)
print("ANÁLISIS DE TRADE-OFF: VELOCIDAD VS PRECISIÓN")
print("=" * 80)

for _, row in comparison_df.iterrows():
    efficiency = row['R² (val)'] / row['Tiempo (s)']  # R² por segundo
    print(f"\n{row['Modelo']}:")
    print(f"  MAE:     {row['MAE (val)']:.4f}")
    print(f"  R²:      {row['R² (val)']:.4f}")
    print(f"  Tiempo:  {row['Tiempo (s)']:.2f}s")
    print(f"  Eficiencia (R²/s): {efficiency:.6f}")

# Análisis de overfitting
print("\n" + "=" * 80)
print("ANÁLISIS DE OVERFITTING")
print("=" * 80)

for _, row in comparison_df.iterrows():
    overfitting_pct = (row['Overfitting'] / row['MAE (val)']) * 100
    status = "BAJO" if overfitting_pct < 10 else "MODERADO" if overfitting_pct < 20 else "ALTO"
    print(f"  {row['Modelo']:18s}: Diferencia = {row['Overfitting']:.4f} ({overfitting_pct:.1f}%) - {status}")

# Conclusiones
print("\n" + "=" * 80)
print("CONCLUSIONES")
print("=" * 80)
print(f"\n1. Mejor rendimiento (MAE): {best_model_name}")
print(f"2. Más rápido: {fastest_model_name}")
print(f"3. Todos los métodos de boosting muestran excelente rendimiento")
print(f"4. LightGBM ofrece el mejor balance velocidad-precisión para este dataset")
print(f"5. Los métodos de boosting generalmente superan a los modelos de bagging (RF, ET)")

# Comparar con mejores modelos de árboles
if 'Random Forest' in model_results:
    rf_mae = model_results['Random Forest']['mae']
    best_ensemble_mae = comparison_df['MAE (val)'].min()
    improvement = ((rf_mae - best_ensemble_mae) / rf_mae) * 100
    print(f"\n6. Mejora sobre Random Forest: {improvement:.2f}%")
